In [1]:
"""Minimal jobshop example."""
import math
import time
import collections
from ortools.sat.python import cp_model
import logging

# Benchmarking data set importer

In [2]:
p_ij = \
"""97	96	27	39	52	67	80	82	97	71	52	92	44	43	43	46	20	82	62	92	60	72	25	65	20	41	14	24	94	75	63	80	60	71	51	53	71	45	49	63	79	24	20	93	42	45	71	21	18	65	40	30	10	68	67	17	73	39	78	46	42	55	66	90	63	100	43	48	53	71	24	21	28	76	87	26	43	17	29	54	99	13	85	74	39	63	35	59	66	72	40	90	100	77	25	22	25	92	61	69	46	59	41	68	82	73	95	56	31	65	15	98	11	13	88	75	44	33	12	40	75	38	73	87	12	68	99	55	22	38	46	41	97	49	81	89	22	12	29	90	36	16	37	63	37	83	44	95	76	36	47	35	22	64	83	29	37	73	15	88	32	40	17	75	96	20	91	55	26	30	53	45	71	21	33	70	98	14	99	20	53	35	93	47	29	98	19	44	28	98	92	47	74	80	48	95	53	22	71	71	26	36	86	73	25	47	96	74	62	61	20	31	60	27	86	87	88	13	12	19	74	40	21	12	21	65	11	87	92	67	82	42	72	80	20	79	69	75	78	40	52	36	53	20	39	23	79	19	71	38	65	58	17	48	86	26	56	71	38	35	59	36	22	65	33	37	65	11	52	63	55	60	35	76	26	21	22	11	32	73	53	13	83	51	14	62	99	33	83	20	24	87	54	81	79	77	82	36	12	20	65	99	82	65	50	15	49	46	39	22	30	69	26	29	61	81	23	46	87	15	85	93	46	48	12	18	47	62	64	24	91	23	31	68	47	57	28	75	84	70	54	30	25	27	20	71	32	10	43	97	36	36	13	53	65	90	29	79	19	89	77	18	99	14	23	70	73	74	88	97	41	84	69	52	67	41	95	97	51	82	94	30	59	22	84	61	90	54	41	34	17	25	48	71	75	51	59	51	37	81	37	64	27	19	18	96	43	12	19	73	57	14	36	100	73	26	85	44	68	44	31	73	63	18	87	64	46	12	18	56	12	95	62	62	46	18	69	91	51	21	54	27	29	20	88	88	52	67	26	79	14	27	71	16	94	29	42	53	27	68	79	29	90	57	10	68	27	79	64	34	24	92	37	63	29	54	46	10	79	14	59	60	71	89	78	19	45	68	29	25	67	42	53	94	95	12	96	22	64	94
93	41	46	23	18	61	13	59	20	67	93	52	98	62	52	82	63	97	10	83	74	86	19	30	76	28	94	94	54	48	66	94	83	17	58	84	81	26	65	48	57	20	93	67	21	30	65	90	14	69	28	11	80	72	93	48	96	76	61	83	94	72	99	93	11	85	59	41	51	90	86	65	99	54	23	87	73	19	64	80	90	90	48	79	13	36	44	19	71	50	45	82	58	14	19	47	70	70	83	10	64	78	80	97	78	18	16	93	79	23	87	70	35	94	59	63	13	41	44	35	83	49	64	65	65	91	79	83	66	60	58	60	12	17	26	71	51	53	80	57	75	51	29	97	64	16	57	20	58	95	15	76	11	95	89	22	51	14	56	69	44	18	96	98	11	71	33	84	20	31	24	59	16	84	68	16	22	58	41	41	56	21	72	81	97	70	52	32	53	36	38	61	13	21	76	93	98	66	77	27	24	45	84	26	67	20	62	31	65	35	90	25	10	31	64	60	12	34	71	15	79	23	62	92	21	92	41	27	47	71	24	50	10	18	37	99	73	38	67	17	58	61	52	50	31	30	35	62	31	73	76	10	52	26	44	86	89	82	25	39	83	30	39	14	45	25	21	66	83	79	58	66	55	30	57	75	100	65	78	51	92	70	73	63	34	27	37	75	34	49	93	98	56	77	69	40	48	55	34	48	12	53	60	70	10	64	52	14	26	66	31	79	34	95	99	98	98	45	57	98	26	66	27	68	49	66	23	33	44	76	31	14	82	11	99	75	64	98	89	56	41	41	46	19	82	42	22	34	49	66	21	71	73	22	59	51	75	35	45	72	89	91	67	77	12	44	75	47	13	76	34	13	99	23	86	75	83	62	88	62	55	94	82	13	82	54	63	88	24	69	47	19	42	15	59	74	22	28	62	72	53	47	41	91	47	63	26	43	22	50	84	96	67	29	86	64	86	87	17	87	17	89	66	97	83	34	39	20	57	26	83	96	28	70	98	27	30	52	26	10	93	14	31	22	14	89	87	33	51	14	27	67	65	31	14	81	39	12	33	78	64	61	25	49	54	51	42	16	33	42	59	75	82	51	86	18	58	50	24	19	75	34	53	79	90	79	51	25	29	78	18	54	63	49	92	98	83	87	100	12
55	79	68	48	48	22	28	99	78	29	47	96	49	69	26	37	76	20	21	12	25	23	71	33	56	40	35	35	44	92	55	20	88	23	23	90	85	22	44	78	72	45	26	91	98	88	96	50	94	78	79	57	82	76	87	12	37	46	17	62	77	72	42	95	12	47	28	27	36	62	30	19	94	97	70	62	21	26	79	82	73	82	89	83	17	66	20	79	12	90	88	51	87	23	85	25	94	55	73	62	93	32	52	10	13	89	99	20	94	31	39	12	52	29	12	14	86	52	55	28	79	52	55	66	53	69	82	25	43	29	60	74	51	49	14	96	90	35	34	30	69	79	64	16	41	44	14	33	85	42	87	65	48	11	69	48	99	13	58	97	36	12	24	41	58	84	40	64	72	37	61	33	37	75	54	87	79	21	66	24	26	69	59	75	19	25	63	90	22	15	89	87	97	38	33	69	83	13	56	99	48	38	68	94	63	17	21	21	85	33	17	70	66	96	45	39	51	15	14	95	11	39	47	53	19	78	26	46	37	62	45	77	31	43	27	10	65	63	31	86	60	43	90	43	15	65	24	59	17	65	79	37	13	58	68	31	66	55	16	96	40	67	96	50	60	93	94	60	30	54	61	56	63	35	11	90	28	24	58	15	54	30	26	96	82	76	81	23	34	65	31	44	43	85	66	63	59	37	40	43	17	90	70	72	51	77	93	26	25	57	78	99	36	99	31	33	97	36	33	26	29	80	45	28	12	39	100	60	36	40	51	37	23	13	61	55	61	24	29	53	44	60	12	45	83	96	34	12	62	96	32	30	37	59	86	19	40	30	88	93	69	44	41	12	73	64	35	99	49	67	36	82	65	94	39	54	40	95	61	55	76	14	52	67	65	96	33	30	12	14	65	30	29	24	79	49	53	36	63	66	49	11	37	81	62	46	47	43	94	94	98	15	61	58	19	56	88	35	50	67	39	26	34	31	71	81	80	74	72	34	90	96	86	77	62	53	99	43	99	23	20	41	74	93	38	94	30	83	66	86	26	98	44	88	92	11	80	87	16	39	77	95	64	19	97	80	73	91	78	86	58	91	23	48	98	73	28	31	79	50	49	58	36	11	94	64	23	42	71	32	97	96	46	49	84	42	97	80	77	16
84	28	52	27	57	97	52	96	27	49	59	13	13	58	70	95	86	65	42	100	97	18	34	67	61	55	100	41	90	26	28	92	81	69	72	85	45	31	79	61	78	28	78	38	89	50	83	81	16	46	75	42	56	19	18	91	90	49	79	86	89	72	85	52	11	57	52	61	26	52	100	35	35	83	54	86	68	18	84	23	54	89	46	44	14	28	52	81	64	68	40	69	76	69	19	100	32	60	77	59	20	88	55	44	68	74	77	88	33	54	83	69	40	97	61	18	73	20	28	35	82	53	62	90	55	41	60	94	79	62	57	52	18	69	79	29	76	99	16	29	70	82	76	39	44	66	67	44	73	26	71	31	13	97	68	30	23	79	75	49	27	89	69	44	22	52	30	48	99	57	45	74	53	41	14	70	82	82	75	87	66	61	38	46	92	81	100	93	90	93	79	25	47	50	41	23	40	86	11	64	13	50	48	56	95	62	50	38	96	11	14	28	79	94	45	34	67	94	26	98	76	54	85	33	62	63	74	43	21	60	95	80	29	59	96	94	71	32	60	38	25	24	98	51	10	39	41	35	77	38	36	48	68	70	36	99	22	34	62	67	97	29	80	79	36	20	38	98	86	37	23	25	87	80	48	37	60	95	70	32	85	52	73	55	66	69	55	27	70	77	70	19	62	24	99	48	43	16	81	47	53	53	22	20	30	59	55	29	96	34	39	71	91	73	66	42	50	29	76	59	27	96	82	35	12	60	21	55	100	91	51	27	19	44	76	29	97	79	80	18	26	70	54	42	78	65	26	76	48	81	42	74	19	100	81	24	57	98	83	36	16	45	66	74	92	15	34	87	42	91	44	50	75	77	38	54	83	62	91	15	59	54	45	42	38	41	63	81	66	18	33	48	99	18	91	39	61	14	52	76	10	39	91	92	37	75	31	95	56	87	43	95	36	72	53	17	55	20	87	58	28	48	40	81	80	35	17	13	45	34	69	60	86	30	91	13	41	15	57	13	25	22	13	18	37	17	14	52	63	42	79	65	59	76	71	18	82	57	21	61	90	31	65	71	68	44	44	30	91	41	12	70	21	69	58	35	22	17	25	65	53	54	19	37	67	16	78	26	92	45	90	42	31	60	21	28	72	10	32	35
53	86	48	57	16	29	45	15	88	82	74	26	17	53	98	30	72	51	48	67	92	10	76	99	63	53	84	81	54	100	45	86	56	41	33	91	95	15	91	51	12	21	87	89	62	42	21	30	69	96	48	76	90	40	37	40	69	14	63	64	62	26	78	60	47	82	51	94	28	55	85	28	44	96	22	36	94	63	83	10	57	64	69	61	67	97	50	79	55	66	85	25	14	59	48	75	11	94	11	29	42	66	31	90	62	62	71	66	61	65	94	80	17	39	72	75	95	12	80	20	58	65	32	89	72	79	31	43	40	68	61	86	90	29	72	13	68	57	87	29	10	98	34	60	43	77	36	40	48	68	79	57	87	33	68	90	63	71	39	24	82	27	54	46	37	46	90	12	77	84	25	79	85	67	34	26	89	86	69	60	99	46	39	71	46	14	27	73	42	17	27	47	86	98	60	96	88	46	75	36	49	86	40	82	15	33	57	50	90	46	41	17	49	18	66	30	82	78	80	62	30	30	71	69	84	17	56	39	37	84	68	70	86	79	59	24	82	47	36	25	79	48	37	90	55	58	78	47	85	46	92	39	100	13	27	21	91	78	86	19	69	56	37	59	50	11	82	55	38	65	75	90	76	55	34	44	53	86	19	55	66	64	16	76	42	96	47	93	19	96	22	81	93	13	28	49	43	92	21	47	48	26	36	90	14	90	68	32	38	36	45	58	36	41	64	19	33	47	76	66	43	70	40	44	95	28	59	92	51	37	35	64	23	20	62	53	32	48	45	34	52	92	31	94	16	19	88	28	48	56	82	12	92	95	62	34	10	96	23	26	68	51	81	27	13	74	72	14	41	12	36	78	89	11	38	17	99	51	22	65	92	73	85	91	84	77	50	13	40	86	83	65	20	36	12	59	43	89	58	71	58	90	45	35	54	72	15	79	29	54	21	94	47	22	79	86	32	35	52	51	74	45	39	67	94	19	48	92	21	78	63	40	24	78	25	92	46	63	72	28	90	28	41	93	71	95	22	28	96	79	18	56	76	98	92	54	47	100	84	23	72	39	87	47	77	73	25	99	31	79	90	67	95	70	49	78	49	22	18	90	77	55	52	30	39	75	37	25	81	99	95	36	75	14	23	79	24	75	90	93
50	83	36	78	98	43	86	90	59	27	21	51	93	21	50	20	12	98	78	62	52	58	26	90	58	76	52	78	27	90	42	92	30	42	55	80	74	66	31	28	53	73	27	23	34	14	24	11	53	42	15	52	62	78	100	10	44	19	76	53	48	38	68	63	67	32	28	41	93	77	46	61	14	72	29	97	25	47	69	66	64	75	70	41	61	44	71	31	23	92	62	72	72	12	91	18	40	89	39	87	45	71	87	75	77	73	73	91	20	41	91	21	16	100	76	39	55	98	87	91	38	20	93	87	60	97	58	25	40	24	87	92	87	52	41	44	26	23	43	49	39	32	25	98	53	39	37	64	88	75	13	43	13	78	66	82	57	77	84	83	74	21	38	57	62	86	39	81	20	77	16	58	22	26	24	22	54	38	21	37	44	92	57	28	66	45	56	97	75	69	63	58	79	82	15	60	24	57	62	93	42	93	66	83	85	83	56	48	78	58	55	61	12	86	81	33	55	77	85	15	90	81	21	78	36	19	65	76	87	84	25	89	46	82	21	72	40	85	25	64	28	19	23	81	82	60	38	51	91	41	70	27	88	55	15	57	21	25	68	12	53	70	29	82	63	41	89	92	58	80	21	50	23	97	91	37	18	55	43	92	22	91	15	45	71	78	68	54	76	43	32	81	47	27	30	48	92	41	60	76	67	92	82	32	94	15	44	94	36	57	49	81	87	51	97	56	16	81	24	43	50	34	33	60	41	33	86	34	94	34	96	91	15	73	41	50	62	37	19	87	41	83	16	58	60	56	17	15	53	45	53	44	25	52	43	85	50	61	88	83	43	20	76	28	76	47	99	23	18	57	64	74	78	91	95	71	38	82	14	74	37	37	54	55	70	22	95	45	95	16	54	77	38	19	12	21	32	59	38	23	94	18	98	52	98	21	72	37	49	40	61	47	38	98	57	93	56	61	11	82	43	73	27	20	85	67	32	34	58	62	78	75	46	25	35	99	31	18	25	65	67	10	51	52	68	57	14	16	28	42	99	74	45	35	60	81	98	80	91	35	88	65	26	90	97	94	33	34	62	47	48	23	75	24	54	86	47	100	72	20	79	30	34	95	17	72	19	96	96	42	50	76	65	70	19	75	61	77	79	31
49	17	86	89	26	93	57	97	10	25	77	48	58	28	80	96	30	43	79	12	35	38	10	16	82	45	85	60	53	95	64	99	87	69	60	51	80	19	32	80	28	88	48	38	53	31	67	36	70	92	63	49	12	76	45	14	82	26	63	57	50	95	49	76	36	59	91	30	71	71	75	81	24	54	88	31	11	100	53	20	20	25	76	83	47	93	77	10	31	97	44	40	22	15	88	63	39	24	100	69	17	79	56	24	37	13	42	98	66	34	86	61	47	60	11	71	15	36	63	83	95	36	41	29	35	41	83	36	83	99	79	59	91	90	20	93	67	23	58	34	70	28	36	86	73	11	30	42	56	34	22	28	96	89	24	41	14	21	76	14	59	43	21	84	72	72	71	66	75	94	34	55	80	62	43	66	88	55	95	86	67	74	84	80	44	87	33	88	86	75	28	44	26	82	45	24	30	14	93	51	53	98	17	68	43	24	60	23	49	27	38	92	77	47	34	16	68	13	22	30	24	27	40	52	99	38	35	93	82	75	62	72	85	61	51	64	12	95	96	40	14	88	30	47	37	97	50	30	15	58	52	41	33	32	23	87	45	10	69	40	17	92	52	26	68	44	46	44	74	96	92	46	98	32	76	37	65	30	44	84	37	93	82	73	16	85	81	54	25	46	97	89	27	78	24	93	100	37	73	22	23	35	38	40	62	45	66	75	11	59	67	17	67	56	100	28	25	47	69	78	57	32	60	87	91	47	43	19	80	35	25	73	46	15	22	67	30	100	73	67	26	95	98	37	37	44	71	49	72	62	55	77	96	56	30	98	15	73	22	56	99	32	12	52	55	77	94	72	24	22	99	93	72	32	51	45	79	33	58	44	27	53	41	74	61	84	21	50	46	18	57	19	51	11	91	71	54	44	99	54	10	67	49	62	51	82	24	29	13	33	44	52	58	71	26	69	62	11	58	76	58	15	90	87	50	82	50	22	78	84	26	44	33	76	27	39	51	34	21	91	32	50	65	73	65	99	14	23	80	89	43	98	42	18	83	77	81	61	92	88	34	29	19	34	46	65	82	59	32	11	67	74	67	56	90	73	51	23	26	74	46	36	34	16	13	85	53	34	63	41	30	50	62	74	66	49	72	64	95	77
73	38	41	86	63	94	96	51	62	27	63	59	68	98	78	70	93	36	53	13	94	75	28	66	29	26	34	97	67	68	91	75	76	59	34	20	66	93	16	47	95	41	23	82	48	70	54	69	92	66	33	65	29	68	65	94	44	70	68	72	84	20	10	62	44	66	42	51	21	19	44	31	81	64	97	46	22	41	51	64	59	34	51	47	86	45	13	78	81	43	87	51	78	31	62	44	66	43	68	34	86	35	10	81	98	88	93	49	48	17	61	19	16	11	23	48	89	90	75	96	60	50	70	98	65	51	60	50	59	31	20	92	96	24	60	80	79	15	33	84	14	41	91	15	61	45	64	13	40	32	26	57	25	45	13	74	62	90	24	70	74	56	43	55	68	83	94	91	91	68	12	83	87	81	24	81	66	74	57	24	75	92	67	42	36	45	48	78	29	46	53	84	37	40	73	11	92	23	78	72	35	37	70	53	43	54	30	47	31	96	77	98	23	84	18	87	75	24	78	38	70	60	36	57	53	34	87	38	57	68	46	91	45	51	24	42	56	35	89	90	81	57	66	97	32	93	99	95	14	94	86	24	55	35	88	73	78	88	50	60	50	36	99	58	71	10	21	64	98	88	39	89	33	71	43	56	81	44	51	73	55	93	84	94	60	95	63	57	88	71	75	83	27	57	34	12	20	71	75	80	79	99	90	64	23	16	43	35	38	41	73	56	79	31	71	86	23	10	78	57	28	46	23	88	49	61	49	11	32	73	58	69	90	10	52	16	50	91	59	13	20	74	41	50	34	76	26	50	76	72	59	94	96	93	32	62	38	52	73	51	44	64	18	88	70	12	95	62	42	63	99	34	63	35	56	36	95	59	28	79	36	10	47	53	45	42	77	28	62	54	81	99	96	15	83	25	70	31	34	17	12	17	35	89	94	64	34	20	93	74	46	86	92	69	82	76	35	71	86	15	14	59	65	17	14	66	66	45	26	55	58	33	12	52	48	73	36	41	80	39	82	81	13	93	97	77	88	24	93	93	89	15	80	82	19	50	72	79	81	93	71	56	49	26	64	89	98	26	98	38	99	99	24	48	38	37	35	11	72	84	65	21	35	98	61	21	40	49	59	39	93	11	83	54	15	63	83	65	87	43
61	71	10	87	62	58	81	24	35	18	78	61	73	93	15	63	22	33	37	47	98	81	60	57	54	73	80	31	47	20	47	12	31	56	50	70	81	93	44	89	37	78	18	88	27	73	21	31	28	43	77	34	93	83	78	21	37	80	83	60	15	18	81	68	71	40	13	51	55	85	56	11	66	41	91	90	68	10	70	30	76	21	94	56	45	81	48	37	96	94	82	92	13	70	31	17	32	42	43	58	32	48	12	65	35	48	63	16	97	68	61	91	39	68	15	99	24	21	20	51	87	89	57	48	67	12	59	22	51	23	57	50	82	32	84	77	49	51	92	40	61	53	25	61	84	88	10	88	70	52	58	88	16	17	18	62	36	92	71	38	54	73	82	83	32	28	22	42	79	23	82	23	17	53	43	27	54	55	28	50	14	72	47	91	32	40	63	33	37	19	77	65	91	34	60	20	75	32	68	95	41	29	36	17	87	43	71	48	90	50	31	39	49	96	55	53	76	65	15	35	45	60	51	12	15	58	16	73	41	75	66	14	88	81	92	98	31	49	51	71	51	40	88	73	58	69	39	26	51	49	85	21	97	54	81	75	37	44	74	75	93	93	13	45	99	16	68	13	35	19	92	52	20	93	29	92	41	33	92	67	24	94	37	83	39	84	10	83	29	88	66	61	59	32	74	31	92	59	90	78	68	33	59	85	48	47	22	90	18	47	83	54	50	65	99	33	64	39	47	19	72	19	94	59	66	28	25	83	19	12	70	46	42	18	34	13	39	46	32	86	29	22	27	71	55	43	88	39	49	32	84	31	45	97	64	27	21	30	24	25	92	62	23	97	79	78	60	87	69	54	26	55	47	72	16	85	44	85	82	12	90	58	11	17	18	45	13	72	94	58	66	24	50	72	66	56	34	64	16	85	63	34	96	60	77	15	98	26	92	18	45	61	29	11	62	54	27	28	32	23	21	56	54	29	93	71	30	37	16	57	37	99	72	91	33	59	10	41	17	100	51	81	96	39	68	34	66	51	87	70	63	15	26	20	18	77	82	19	50	56	46	36	50	56	78	13	77	89	43	91	52	61	57	97	24	58	51	87	53	23	27	51	61	60	96	29	61	95	13	38	60	57	22	55	24	46	30	18	97	57
91	98	99	62	78	50	85	36	94	84	94	19	38	30	22	87	42	29	37	91	10	63	12	53	11	29	21	82	28	73	48	77	29	78	26	58	10	62	76	56	37	67	10	54	91	81	75	17	20	99	60	40	87	65	27	23	50	36	48	39	99	54	50	73	92	74	97	37	66	24	69	44	85	43	94	15	45	13	61	30	93	92	91	79	27	17	18	59	66	19	40	57	24	79	20	30	89	93	90	26	37	67	62	60	72	35	33	53	64	24	52	21	43	57	65	21	93	33	31	13	83	64	35	18	38	63	48	85	45	47	94	73	39	67	62	94	87	83	27	12	61	27	33	49	70	89	96	77	44	33	51	64	60	78	39	27	89	77	19	52	81	28	67	59	98	70	10	54	44	38	42	97	59	51	32	89	91	80	19	71	95	64	75	47	62	31	48	82	31	37	82	21	19	35	69	17	10	50	86	25	20	99	99	45	87	23	51	13	44	65	64	72	99	47	94	81	18	67	77	19	26	69	73	24	90	49	49	94	100	67	26	82	39	33	38	87	71	49	57	15	69	59	46	95	10	74	33	21	92	71	87	23	91	83	19	16	52	70	82	43	69	46	93	50	67	23	72	35	81	87	86	24	21	30	83	96	91	48	62	82	60	38	84	34	71	84	82	28	53	19	71	84	85	56	32	60	89	14	53	65	55	57	80	83	10	63	11	96	49	56	74	76	75	53	52	56	25	66	27	26	71	31	35	85	54	87	11	19	89	43	33	65	88	17	86	68	95	79	41	93	95	80	18	94	14	99	87	86	77	98	18	50	27	52	44	47	16	56	48	19	26	25	50	76	31	28	26	25	90	48	49	72	77	89	100	74	13	73	14	67	73	61	48	38	73	80	45	26	75	87	68	75	96	12	72	14	84	57	59	20	51	48	54	60	45	13	83	51	67	63	35	57	79	18	28	77	93	53	87	42	42	82	89	32	52	86	64	93	16	18	65	16	12	43	28	70	63	80	26	80	25	28	16	36	38	24	95	98	30	34	62	55	30	72	12	50	74	78	27	50	70	69	72	98	43	40	64	68	62	36	36	12	14	51	84	46	61	33	61	53	14	90	88	18	46	13	63	51	27	36	15	95	46	65	43	83	72	91	21	45
70	92	88	23	66	52	33	14	47	91	51	74	52	82	88	31	20	37	77	89	98	97	11	37	32	65	88	27	19	54	25	34	88	54	58	36	28	61	65	84	18	60	42	88	85	30	98	54	47	72	97	20	90	78	74	26	21	66	14	10	50	15	93	87	35	20	54	92	67	62	63	88	86	99	50	66	78	40	72	18	39	68	70	25	35	61	15	14	100	74	12	26	60	90	51	55	86	63	27	44	13	16	11	35	95	27	94	14	69	12	87	80	12	83	77	82	61	91	26	90	73	71	30	49	24	37	23	96	63	63	30	34	41	26	94	93	12	98	83	40	95	79	40	23	72	23	77	32	70	39	16	81	81	94	62	27	96	52	83	51	85	73	17	34	97	55	14	80	26	79	60	47	65	16	24	80	82	51	28	60	93	16	94	74	97	88	31	46	14	72	55	82	77	94	56	42	30	56	40	97	42	94	71	61	31	63	44	91	20	87	72	44	73	26	86	65	81	33	23	40	43	60	59	11	31	18	94	30	88	42	18	20	38	59	12	21	60	22	13	17	81	26	42	44	85	65	75	20	47	53	24	73	68	25	32	66	55	11	99	42	65	86	91	44	12	97	77	86	23	48	27	17	81	21	71	86	67	54	85	80	87	18	64	48	64	57	56	46	29	20	18	17	34	41	65	22	14	39	95	33	12	95	64	32	16	79	35	50	70	30	13	53	43	70	63	37	50	49	58	93	43	97	56	24	97	56	90	68	16	72	97	27	43	25	13	49	87	81	61	25	21	77	26	19	91	46	16	74	80	71	23	67	21	64	27	11	31	47	30	25	46	28	98	58	52	64	23	25	61	35	13	84	40	90	98	74	68	13	91	21	40	28	80	87	69	79	37	78	49	11	25	52	69	69	38	48	97	78	19	64	33	72	83	95	13	93	19	83	85	38	88	73	79	79	31	18	22	95	83	24	26	58	26	99	54	59	87	29	89	82	12	18	96	75	97	62	32	29	25	72	95	21	39	45	43	37	24	36	21	98	43	53	50	20	13	84	37	27	95	75	51	99	93	44	81	36	10	66	12	99	86	69	51	23	95	67	64	38	13	32	64	70	94	51	52	95	13	76	16	32	48	19	31	48	91	56	72	88	76	40
81	34	83	26	23	21	94	62	61	18	17	33	84	35	27	95	24	17	12	27	59	78	71	77	43	85	71	44	27	65	68	28	90	66	64	100	61	26	37	60	95	41	26	48	18	37	33	30	95	64	49	28	30	37	73	84	85	100	53	25	92	67	77	19	35	30	29	80	71	45	98	27	81	34	96	87	11	30	42	53	100	69	98	53	59	23	24	57	27	74	96	25	82	77	24	37	55	67	79	79	71	56	21	24	38	23	47	87	13	46	54	20	28	17	69	75	96	17	56	34	62	90	45	31	90	79	82	56	34	50	76	26	48	91	79	82	35	70	50	33	68	74	85	61	38	64	69	17	28	11	74	25	65	61	92	96	42	58	76	70	64	24	87	15	52	15	55	31	15	37	39	48	49	48	79	38	38	12	23	25	32	11	47	95	97	23	18	65	28	41	28	17	79	83	90	80	27	61	39	54	12	37	69	97	99	27	33	85	34	65	40	28	90	29	14	55	34	41	20	32	85	81	91	67	62	90	54	85	27	58	41	90	24	32	78	69	71	79	91	22	75	13	55	18	100	36	54	67	30	12	71	15	31	60	90	44	16	64	93	93	57	68	71	92	67	49	94	97	55	72	26	91	27	24	14	37	30	38	48	96	18	12	63	85	63	99	60	21	27	56	34	18	92	74	99	62	17	19	62	73	75	60	13	53	12	16	36	100	85	31	79	41	29	69	58	85	84	65	16	27	52	82	74	39	42	85	94	53	69	56	19	25	63	74	17	28	38	84	91	61	36	46	14	51	37	80	74	47	57	85	34	78	52	16	61	45	46	71	93	91	54	98	45	41	60	73	58	89	86	84	49	99	25	84	47	78	37	73	37	81	31	63	41	96	39	44	17	81	72	94	85	53	59	29	44	12	96	85	34	90	35	86	92	79	57	90	83	28	59	23	62	91	65	31	41	85	32	18	72	72	21	36	52	22	23	72	53	14	41	11	59	84	16	28	34	89	80	94	46	50	15	75	82	74	42	22	72	24	43	93	48	13	55	69	60	20	80	65	11	68	33	56	59	56	97	87	21	64	60	40	36	62	10	63	88	64	18	40	56	60	87	48	70	100	90	10	79	78	57	38	25	53	89	50	53	31	51	76	82	93
39	92	33	71	53	60	58	88	65	85	86	83	53	32	29	96	17	52	77	58	30	96	36	48	99	45	29	46	79	51	44	48	68	86	34	69	49	84	19	69	55	81	74	40	19	57	39	71	11	49	38	26	67	48	98	33	47	61	66	43	37	14	56	42	23	85	85	32	90	98	33	20	28	99	96	12	36	46	16	97	83	41	68	42	40	89	22	45	14	26	58	43	17	62	70	57	31	86	74	69	20	22	94	84	22	67	42	96	92	29	51	31	36	72	69	47	95	82	87	65	57	37	83	14	11	64	34	25	53	17	51	26	41	77	79	27	34	56	67	28	95	61	53	34	13	43	65	62	71	16	100	28	14	85	36	74	76	42	31	85	80	42	39	63	54	84	43	32	50	22	25	11	44	60	62	92	81	74	72	95	99	88	13	51	31	93	85	81	35	98	23	16	46	60	77	50	57	83	81	63	42	30	40	83	27	92	40	86	90	35	41	13	71	22	91	16	69	17	98	74	35	29	31	81	12	38	92	88	72	42	14	62	32	22	33	34	62	94	88	14	14	36	33	48	13	50	71	57	28	57	91	57	60	81	29	39	89	27	51	58	57	72	17	69	44	87	95	39	79	91	59	47	83	82	70	94	100	19	63	77	28	14	65	74	36	89	39	31	87	94	81	47	95	33	35	12	91	56	59	56	83	95	90	15	76	38	61	37	53	75	39	41	61	86	54	100	52	46	59	44	87	47	100	32	47	92	73	53	61	83	95	55	28	30	94	95	79	62	10	87	45	82	63	19	55	30	21	90	13	31	96	47	39	36	81	85	66	14	12	12	70	16	25	53	73	15	74	14	77	67	32	35	82	20	61	35	49	39	57	49	47	21	70	92	63	20	69	33	37	38	14	63	51	12	86	24	29	79	24	13	80	24	61	51	95	29	77	45	48	94	63	80	64	75	21	20	100	20	66	41	40	35	49	83	74	11	21	74	18	96	31	37	33	45	64	61	49	29	38	43	27	78	97	37	44	51	91	60	69	26	51	82	96	76	55	91	93	70	84	69	92	63	78	26	37	98	59	87	39	45	50	54	90	19	13	13	74	40	34	21	93	44	63	61	27	76	90	99	43	66	85	24	95	11	38	72	64	96	95	72
76	32	26	97	38	38	70	90	10	36	51	57	40	17	28	24	94	25	95	62	63	60	38	61	27	43	47	26	25	76	20	67	89	75	20	11	16	15	22	89	16	13	29	95	46	54	81	33	28	11	66	98	85	55	64	47	72	42	35	20	79	100	25	40	75	55	77	100	92	24	35	25	82	52	92	83	67	39	99	13	12	76	66	50	84	12	75	31	59	91	77	84	24	32	29	31	22	93	62	45	83	84	85	81	25	75	27	59	63	46	26	35	96	74	62	57	19	57	83	15	84	56	99	77	99	86	56	36	49	46	98	16	43	17	95	45	12	90	39	61	59	30	70	29	39	67	33	19	51	26	56	50	42	18	57	22	98	83	77	44	70	39	55	15	100	99	49	37	29	31	50	14	86	32	19	37	51	45	21	34	93	68	24	26	70	84	88	20	67	60	45	36	93	15	43	50	13	31	88	18	24	100	51	70	29	29	71	51	64	65	26	75	59	39	97	20	26	11	63	79	97	66	67	23	95	38	25	78	39	74	70	75	94	95	35	28	60	11	90	80	36	55	52	75	42	99	30	44	77	56	77	39	92	75	22	32	71	91	53	72	51	51	62	84	26	93	77	49	81	85	82	66	86	96	61	23	44	75	35	19	91	52	55	74	98	95	95	61	50	57	82	50	41	99	82	94	40	31	83	48	93	91	73	21	22	37	77	30	62	68	80	97	72	24	84	79	100	21	59	22	71	29	86	77	70	45	16	51	64	80	20	43	66	93	85	32	37	87	14	89	14	30	24	84	95	32	47	27	46	21	64	46	83	58	78	68	19	33	29	80	71	16	16	17	100	31	22	15	63	93	61	11	77	70	27	84	28	60	68	81	37	27	18	36	16	56	80	87	30	25	75	23	76	46	96	40	71	76	88	81	78	80	78	31	84	39	54	32	34	98	41	12	51	52	39	22	50	23	57	13	35	66	88	12	91	42	100	29	28	67	95	55	51	42	24	68	54	66	62	73	47	26	61	16	17	48	38	29	70	49	73	60	98	49	27	66	41	39	53	50	78	34	72	72	54	29	43	67	16	29	37	41	11	65	41	22	38	70	44	57	94	64	70	15	55	47	77	79	57	44	41	21	94	37	59	99	97	99	96	41
66	71	21	71	43	80	53	97	66	63	29	51	36	67	42	38	85	12	59	78	70	71	92	86	42	79	59	31	39	27	61	81	98	35	96	28	23	45	54	94	79	59	98	41	82	37	95	99	58	93	84	99	62	41	96	55	98	17	25	32	48	66	67	88	35	82	65	85	11	38	81	94	40	61	50	93	91	75	85	57	20	94	10	51	53	49	20	19	75	74	97	81	29	66	11	15	38	68	75	40	18	73	52	16	67	40	53	36	53	16	53	25	80	25	22	27	93	71	54	18	48	58	59	48	27	47	72	62	18	27	21	77	90	25	50	70	28	60	92	53	80	84	56	75	77	47	22	13	13	75	67	34	13	23	12	87	74	78	93	98	96	64	37	94	57	97	19	18	82	20	11	71	76	64	48	42	15	50	51	80	78	47	48	14	76	25	89	64	78	65	91	86	19	47	31	96	38	68	47	71	40	63	58	38	97	27	69	44	19	64	57	21	29	14	88	59	70	95	95	24	91	99	93	77	92	43	16	84	37	75	75	74	59	95	71	91	69	72	88	31	27	66	94	56	84	40	40	62	83	15	31	44	87	19	81	44	80	16	72	61	28	23	31	37	41	56	16	74	56	91	21	77	99	57	91	20	75	75	26	56	52	37	19	78	60	36	66	24	95	19	42	37	51	39	13	48	95	30	37	55	99	45	13	39	91	72	62	45	40	58	19	74	33	44	85	52	62	23	54	86	89	33	87	68	40	21	30	55	51	33	81	35	79	68	30	90	57	93	59	19	90	21	31	15	73	26	17	36	86	70	95	37	61	95	79	76	71	89	40	53	98	62	86	76	50	82	79	82	44	96	89	12	40	27	76	32	65	27	74	32	97	56	34	21	80	69	11	71	96	33	62	24	47	12	95	85	82	45	54	73	67	19	10	38	11	83	35	17	92	95	19	32	87	56	53	36	17	93	60	71	24	69	22	45	95	54	54	11	47	15	24	85	46	81	87	67	58	21	16	100	100	52	40	72	92	51	53	53	18	96	44	96	14	24	75	96	55	78	58	50	98	30	24	83	72	16	34	50	44	57	80	94	25	22	65	79	57	78	90	81	63	93	10	23	85	67	59	54	31	26	60	93	15	42	90	61	94	63	81	59
32	83	23	14	12	11	35	85	85	72	92	70	80	54	91	65	53	40	88	35	57	56	94	61	25	48	25	83	61	31	33	88	42	80	63	25	56	30	95	50	41	26	30	15	65	66	59	31	19	69	39	68	35	79	74	28	39	71	50	42	67	69	91	95	14	10	73	73	15	30	68	69	99	75	94	52	26	14	79	47	77	78	46	88	47	30	30	42	39	65	93	13	27	73	66	87	47	57	35	47	59	44	41	63	90	66	21	27	19	57	32	40	84	12	39	43	47	98	80	36	81	42	55	74	78	79	17	57	41	81	11	59	38	75	79	55	72	51	61	60	97	72	68	76	24	79	58	87	17	81	81	88	41	89	58	45	90	71	25	67	31	73	57	72	51	34	21	100	62	77	41	80	72	28	31	65	82	56	29	52	67	75	92	54	55	63	20	71	25	85	99	77	75	46	84	92	73	35	78	52	94	11	46	54	78	93	88	99	49	59	81	22	87	72	96	34	34	98	84	14	88	72	26	77	61	93	28	100	26	95	41	24	39	43	82	29	83	12	42	88	60	65	57	22	24	95	60	68	34	59	27	74	25	24	32	54	48	68	49	26	68	41	44	99	29	36	36	49	20	80	19	87	91	17	71	43	93	56	55	70	48	12	38	87	97	82	29	46	68	54	42	18	69	32	54	13	89	30	43	11	61	24	23	83	32	69	33	45	55	65	22	68	13	58	81	90	82	91	19	90	69	59	39	75	24	34	97	20	24	60	52	13	12	31	20	92	74	81	11	62	33	50	21	43	42	87	42	19	93	98	79	78	51	98	57	94	89	31	58	13	71	25	87	43	88	26	59	25	57	30	33	86	97	63	62	12	41	45	90	33	46	72	53	60	42	48	83	99	77	30	44	57	15	21	35	50	43	27	76	31	85	24	68	74	76	50	13	67	69	13	98	72	96	23	48	11	44	26	34	15	95	97	72	64	53	70	64	30	38	14	79	13	42	18	30	68	59	67	15	57	72	28	59	39	70	53	71	43	42	89	72	96	19	32	57	35	31	59	24	39	92	44	52	68	10	93	19	24	45	52	33	53	54	86	63	52	97	15	63	41	37	50	73	67	16	48	74	82	36	73	35	38	99	60	98	95	83	75	33	98
45	57	48	95	27	98	82	64	37	28	35	78	75	99	77	81	69	83	36	66	95	71	12	39	37	62	89	48	89	47	22	21	55	49	77	89	53	72	83	75	11	26	84	51	76	43	93	70	68	84	63	18	47	38	77	47	71	59	49	15	53	16	90	66	87	94	94	37	88	11	42	84	50	41	28	65	36	53	21	87	40	95	84	16	62	53	55	39	10	89	51	64	52	73	18	98	79	77	84	55	35	95	14	18	44	48	39	71	25	31	49	37	74	66	65	59	98	25	64	30	41	94	32	24	81	38	24	63	59	75	90	78	53	22	54	93	26	45	92	68	59	90	30	59	62	19	72	27	40	75	47	92	92	98	97	91	83	12	70	60	28	87	62	20	88	96	22	75	59	35	69	17	78	58	35	48	31	20	68	25	62	34	69	17	19	52	11	33	89	91	92	92	82	11	92	72	98	68	72	16	29	45	79	40	86	92	38	91	53	25	17	43	72	67	98	21	98	96	30	23	41	53	28	33	12	23	40	90	17	76	35	69	90	45	21	15	32	90	25	93	14	36	85	88	70	60	13	90	72	67	15	98	73	89	12	30	12	93	16	11	97	72	85	64	99	52	64	20	92	53	82	68	99	54	62	19	93	54	55	28	15	88	11	88	17	23	27	69	62	51	40	95	98	84	71	21	28	16	78	58	12	41	40	28	23	90	33	95	26	10	24	72	20	15	31	65	30	97	80	94	26	50	56	27	52	86	44	22	67	67	16	87	34	52	41	77	41	99	13	93	56	63	89	85	75	89	12	21	49	73	57	59	30	23	94	21	66	48	61	24	40	55	45	50	10	90	69	39	13	99	91	59	20	36	52	96	43	22	98	27	44	74	40	17	85	41	84	97	12	50	65	31	56	22	25	19	10	36	32	38	31	26	77	46	81	29	18	67	91	45	41	21	19	96	45	18	91	16	78	12	65	10	71	20	89	32	35	59	77	22	60	78	42	15	85	50	73	48	17	28	52	77	54	33	70	26	91	39	10	38	46	61	47	93	22	90	65	20	65	18	39	23	56	58	60	92	93	32	41	95	39	98	19	77	93	27	26	33	12	86	45	65	13	23	11	55	27	92	52	20	63	79	81	74	82	98	49	84	13	87
35	48	95	84	82	76	62	54	65	49	83	86	56	32	27	96	96	65	85	62	26	10	83	16	64	63	34	61	57	85	88	37	32	32	35	11	24	95	27	57	52	96	86	82	35	56	80	87	42	94	68	41	16	65	26	95	97	39	80	84	79	84	24	41	85	15	94	73	30	72	39	80	99	40	32	68	81	68	27	39	98	27	43	97	14	31	54	71	14	34	37	41	54	38	15	83	33	13	69	72	10	84	23	85	60	71	57	87	12	90	15	77	84	96	25	46	26	92	83	20	12	38	37	95	13	99	64	83	81	40	54	25	73	26	55	29	50	19	49	20	93	39	23	26	74	21	66	54	67	37	65	43	100	38	16	74	90	78	100	83	69	99	77	19	22	27	75	29	58	13	85	28	12	73	25	76	32	35	53	47	52	83	73	30	59	47	44	80	62	31	50	98	77	89	58	25	99	18	37	95	12	49	20	97	69	66	28	20	47	51	46	54	96	44	92	78	75	63	29	91	100	60	66	21	22	72	43	99	69	59	29	46	43	11	69	81	24	25	81	28	64	93	83	74	65	19	78	76	28	88	99	80	54	68	20	82	21	22	38	69	39	19	77	59	99	55	94	82	11	80	26	63	47	69	30	11	13	12	18	26	54	79	64	89	16	42	29	24	16	13	26	61	83	21	21	20	75	72	14	53	18	12	85	34	10	89	95	26	49	63	68	91	71	77	27	45	21	68	24	88	39	74	59	27	49	52	68	13	93	55	91	37	15	73	39	60	99	12	18	96	58	46	26	16	23	83	31	85	100	58	32	45	53	43	55	18	91	54	98	78	20	20	57	65	43	48	85	59	74	54	66	41	69	44	48	13	45	18	26	66	83	36	12	55	77	14	70	38	60	64	41	28	97	58	34	60	62	66	94	71	37	57	93	17	42	36	55	22	55	68	93	59	59	23	44	49	62	56	64	62	50	34	25	37	95	46	27	63	32	81	55	81	33	97	11	69	58	48	54	47	19	44	77	93	74	97	74	66	76	26	87	73	18	51	56	85	81	94	19	70	44	27	52	80	47	69	48	64	85	29	72	16	76	78	82	73	77	24	19	22	97	44	71	33	70	98	84	60	13	22	59	83	87	100	12	73	81	44	95	39
34	13	39	69	25	19	99	67	56	66	15	71	69	93	41	89	20	98	41	26	15	61	35	54	59	46	91	62	34	42	35	45	86	92	75	94	21	58	40	13	97	25	82	71	44	89	92	59	62	88	18	66	41	12	84	51	44	72	67	69	72	53	30	82	30	49	64	26	66	74	77	44	39	66	16	90	57	82	100	62	24	22	94	16	65	75	88	56	36	19	27	44	47	29	93	23	46	32	17	25	91	39	13	78	87	98	34	19	23	69	21	12	63	64	60	90	96	83	30	15	88	13	28	80	12	87	11	51	12	97	62	25	28	25	59	35	20	14	18	83	45	73	31	11	59	92	17	86	96	60	32	74	32	43	60	96	13	96	71	75	56	94	70	72	13	88	78	18	61	58	49	84	56	93	34	14	89	83	67	38	44	20	27	53	41	86	64	67	37	15	52	47	35	87	58	62	82	75	36	53	67	16	51	40	60	24	12	29	80	91	38	31	69	95	65	82	12	20	60	26	48	39	72	52	24	96	65	27	89	90	20	70	40	83	99	74	19	97	88	83	33	41	76	14	49	56	77	22	31	72	45	26	61	97	28	71	97	82	94	26	84	98	86	92	79	96	92	37	95	18	72	81	67	86	97	50	79	32	40	57	96	77	54	49	77	23	82	61	13	31	76	48	43	76	87	66	39	89	100	43	43	81	64	34	88	68	21	30	63	36	27	58	86	83	49	32	82	38	59	32	22	60	85	41	56	15	47	32	49	94	93	70	65	12	42	100	82	33	43	36	55	94	79	63	27	73	69	66	90	84	27	95	26	52	100	40	69	78	77	60	27	69	83	83	98	69	41	57	19	14	36	30	99	60	43	89	31	70	92	16	37	94	91	26	84	83	48	23	53	15	59	12	17	82	72	25	33	25	37	36	14	20	65	19	22	35	95	87	24	30	44	31	45	24	97	99	51	39	91	51	72	28	94	71	57	46	29	31	54	31	97	27	49	54	47	97	79	31	15	73	94	38	15	19	10	33	11	42	25	91	100	91	61	96	52	44	11	32	29	54	45	12	57	35	71	33	64	85	25	83	15	93	67	28	98	62	91	90	87	27	46	92	74	52	29	46	88	16	67	57	82	69	65	69	40	45	91	30	64	45
13	67	60	60	49	81	86	60	94	99	88	95	82	59	74	33	87	97	90	34	19	57	60	90	96	13	55	87	35	52	80	16	84	65	36	78	10	50	72	31	82	44	52	55	73	19	58	64	89	17	92	58	57	40	48	18	10	60	21	94	26	66	37	71	46	58	67	41	63	16	40	54	82	50	17	81	73	39	19	65	29	96	36	59	75	82	66	60	94	41	68	20	77	84	94	23	33	74	31	54	92	53	24	91	79	75	14	26	93	38	95	33	34	40	30	30	61	12	62	41	84	84	49	18	12	91	37	35	84	72	85	59	57	49	60	95	49	53	66	22	36	77	66	33	31	84	56	19	44	93	13	12	19	19	57	83	58	55	44	92	64	70	46	21	76	43	16	70	65	35	40	62	96	66	75	81	56	94	33	80	59	81	26	30	60	84	90	70	71	43	85	21	33	24	31	55	27	29	91	85	52	90	19	68	38	57	29	94	94	56	39	39	55	89	23	28	36	26	57	22	96	92	31	93	49	55	79	74	38	19	59	70	28	38	61	38	97	27	60	12	42	74	75	77	29	78	97	95	78	34	37	57	39	43	60	87	94	14	99	43	28	41	57	73	12	30	10	37	67	10	63	99	95	30	74	77	52	42	18	25	50	72	51	59	60	96	44	94	18	44	56	49	100	36	27	47	36	91	26	16	92	71	87	69	28	45	85	52	45	51	67	83	66	26	52	92	57	78	24	84	58	96	82	56	25	14	69	31	37	31	33	94	55	40	29	68	57	16	12	43	32	51	38	52	63	18	63	63	82	16	26	69	32	59	50	96	48	80	81	24	53	94	28	84	91	78	78	26	68	40	53	35	96	71	57	63	39	79	98	11	97	75	28	88	23	28	83	79	15	61	57	100	46	90	50	39	71	85	78	46	62	30	61	13	30	34	32	93	10	91	60	86	43	19	21	86	75	83	31	32	34	54	50	31	24	22	35	19	11	25	46	20	96	30	58	22	32	53	88	44	36	60	28	89	50	22	77	16	49	92	57	78	16	81	20	70	87	84	72	48	69	36	14	74	19	98	65	19	44	41	33	35	31	14	36	41	62	50	99	63	70	15	92	81	48	28	43	80	66	10	93	55	46	35	47	72	99	40	44	12
93	86	22	84	59	32	39	95	21	67	71	67	95	62	92	50	98	35	17	83	45	80	38	54	77	22	20	87	78	42	97	79	51	11	26	92	40	86	66	93	22	44	98	95	63	78	18	45	90	26	52	49	78	50	62	69	73	39	78	52	38	82	67	28	13	100	56	48	45	11	75	44	79	16	91	56	39	61	84	99	43	83	75	58	31	21	48	52	64	56	54	19	33	83	73	68	25	23	20	80	58	51	97	90	30	22	37	82	85	17	92	71	79	32	83	61	42	73	77	43	32	100	84	24	21	94	60	25	78	55	64	81	92	93	12	99	61	88	46	72	76	34	97	23	75	39	40	42	76	91	47	53	59	20	13	91	23	54	79	68	20	53	50	81	92	80	52	90	31	11	65	27	70	15	94	62	51	23	93	49	84	94	66	90	57	50	53	53	59	20	26	69	37	80	35	89	79	39	97	41	80	70	23	29	13	48	88	77	47	99	24	45	47	86	89	50	25	21	64	18	91	13	51	32	72	62	30	49	75	17	97	16	65	67	95	37	83	53	43	85	11	26	37	88	84	88	28	81	90	97	23	82	74	34	37	89	28	27	80	73	19	48	69	14	87	97	42	45	60	96	71	11	69	75	53	39	88	79	67	15	69	19	12	14	19	70	23	41	40	18	85	48	57	47	92	30	60	62	91	72	26	85	17	50	26	48	40	86	29	44	55	59	47	63	21	12	24	22	96	94	78	25	23	92	41	18	49	16	70	27	67	95	34	72	30	15	98	41	97	72	84	30	56	54	98	84	55	50	20	80	62	25	74	38	20	92	86	39	13	43	96	17	16	66	79	19	48	76	50	56	20	32	37	61	53	63	84	68	55	32	58	47	32	49	39	58	79	97	24	81	27	48	57	16	13	23	50	66	41	23	77	90	18	59	28	11	93	47	44	77	67	71	16	60	36	48	45	21	83	22	49	97	55	19	20	61	98	20	56	16	68	73	23	12	48	85	63	78	57	60	85	30	67	95	53	11	78	42	11	22	38	62	67	65	97	34	47	29	25	48	76	66	40	11	23	67	71	68	19	31	84	69	14	67	56	32	23	17	18	59	100	44	52	86	72	16	54	38	89	69	66	77	80	39	54	41	84	20	96	92
67	80	94	63	29	20	15	14	78	46	52	38	96	29	56	66	39	40	88	11	61	94	36	83	56	39	61	73	74	67	36	55	78	38	63	14	53	80	48	52	77	47	55	66	66	77	75	49	10	11	40	78	15	47	29	77	46	34	54	58	28	32	65	59	17	71	82	18	91	67	72	42	46	88	91	56	28	88	90	49	50	33	31	49	17	87	95	72	23	19	21	87	49	20	93	61	13	31	19	33	60	89	43	29	87	93	93	93	52	85	89	76	22	32	27	43	24	53	98	87	20	85	21	18	85	98	47	90	91	19	58	90	37	43	56	66	50	94	63	28	47	26	61	37	75	87	59	68	48	64	71	22	54	44	96	23	85	26	72	89	55	19	57	52	18	87	89	98	97	22	82	52	27	94	76	24	62	82	59	24	77	37	40	61	28	29	28	42	56	93	77	48	87	33	41	28	23	19	95	14	24	92	67	36	48	63	36	36	51	80	72	24	27	71	18	24	51	26	79	88	15	11	27	69	38	31	48	21	48	90	38	28	21	41	83	70	81	90	59	63	94	10	32	74	93	66	67	45	37	45	87	51	77	25	92	90	56	30	80	67	40	33	62	23	62	26	22	45	62	72	15	67	51	35	47	50	54	98	56	41	40	54	85	13	97	20	18	43	99	48	23	99	19	42	72	57	37	59	63	87	59	30	30	11	52	75	56	70	22	22	37	72	47	59	30	14	98	40	75	28	37	85	46	48	44	25	32	27	40	38	13	33	97	98	95	81	68	82	20	58	11	16	46	11	24	89	87	37	59	89	91	12	78	99	13	68	56	93	73	96	68	24	98	81	41	38	15	57	59	80	65	93	37	18	13	44	82	67	24	20	61	96	31	49	79	54	16	37	66	29	57	99	31	94	23	75	95	76	23	68	39	86	48	95	15	48	71	54	71	75	26	63	52	31	24	93	58	47	73	61	38	47	99	71	55	97	20	31	29	90	32	22	63	31	27	65	30	98	30	29	16	35	72	32	11	52	49	17	35	51	83	44	95	97	46	15	76	51	78	98	18	96	81	64	11	65	42	61	43	98	17	35	74	26	68	98	57	88	40	47	15	44	83	31	67	87	97	20	99	10	50	52	85	61	60	15	16	32	10	51
55	69	16	96	91	85	75	99	11	30	45	41	62	96	15	33	22	88	50	57	75	88	70	40	65	85	70	62	47	98	47	61	50	72	90	64	39	99	61	64	81	59	95	92	45	61	95	86	67	62	35	90	58	44	29	14	23	69	39	47	38	84	16	62	48	24	35	19	39	69	14	18	31	91	59	65	10	79	23	29	35	55	97	60	75	73	49	42	37	87	69	46	26	59	30	67	92	96	17	72	67	73	92	76	74	78	34	72	21	16	97	42	58	61	59	14	35	29	96	26	88	25	23	40	76	27	97	28	21	34	62	31	84	49	73	12	46	20	19	51	55	25	65	54	45	85	20	47	15	66	64	25	38	38	43	58	55	83	59	82	75	31	67	78	99	51	80	63	73	16	12	41	50	21	36	61	68	10	71	47	56	32	38	42	69	71	60	69	59	58	52	46	97	72	38	82	83	60	79	59	57	34	15	42	70	64	88	56	18	76	59	100	69	96	62	60	56	46	99	36	48	81	90	25	20	77	91	21	37	39	17	35	38	62	68	13	62	40	90	32	97	85	11	70	60	59	74	44	24	86	74	28	51	18	95	72	56	20	98	63	81	11	95	34	69	51	34	94	33	40	64	43	46	59	62	47	79	28	56	49	57	14	15	96	33	26	97	60	50	18	90	37	54	58	27	52	84	79	37	65	67	30	97	62	78	90	18	34	31	34	38	80	68	58	59	49	50	88	55	99	66	60	38	96	54	59	50	74	17	28	36	87	71	16	75	91	86	42	68	73	48	10	30	57	95	70	78	80	20	21	87	20	68	82	66	19	71	54	52	60	86	87	39	77	62	48	64	97	19	51	45	42	49	49	39	12	19	43	56	29	42	84	11	79	100	13	20	60	15	82	54	31	84	69	61	63	85	78	10	86	24	87	55	73	55	57	16	93	16	37	31	23	61	51	95	89	97	31	69	98	65	21	68	99	43	39	59	83	90	42	78	19	86	22	88	32	13	47	84	18	85	19	50	28	64	71	39	38	62	10	78	91	86	60	25	46	98	49	21	59	46	17	15	28	34	70	41	15	29	90	30	89	55	85	62	27	72	83	32	64	31	75	68	11	66	62	65	58	80	63	79	75	28	65	48	48	94	78	70	18
31	79	41	50	71	61	12	16	26	87	86	88	45	61	89	92	61	42	99	48	23	81	87	72	51	52	94	70	100	23	84	59	86	55	61	47	76	76	15	29	36	89	59	71	25	26	72	83	86	16	38	51	65	34	96	37	61	72	54	43	56	65	14	74	68	71	36	80	47	89	32	67	16	84	26	61	32	16	62	55	90	92	94	43	66	75	33	66	92	13	67	54	52	89	28	56	60	30	63	84	11	13	89	76	53	25	65	86	90	100	21	76	73	24	44	90	26	68	99	52	96	24	91	13	52	54	45	66	71	38	54	11	54	27	88	39	66	65	29	49	13	38	76	50	30	32	52	79	56	14	36	37	43	77	41	26	32	61	86	88	76	52	67	33	57	65	89	32	49	33	59	84	38	75	66	52	47	36	99	41	22	97	89	33	92	57	66	56	45	19	61	32	23	19	49	58	30	67	11	43	31	86	91	15	49	82	59	61	82	43	78	81	34	84	32	62	94	22	26	31	61	75	98	89	38	79	40	79	82	72	65	42	52	31	54	34	70	60	86	79	20	65	97	58	51	45	52	73	69	19	56	35	69	49	89	93	59	29	81	56	74	46	28	73	33	78	36	27	85	13	28	17	29	60	57	96	58	99	24	40	11	14	87	67	23	32	23	24	84	99	74	80	37	18	71	85	52	32	15	24	42	16	56	60	18	86	23	71	82	12	76	79	52	62	53	93	65	18	84	75	28	95	92	22	23	68	83	53	42	20	25	44	51	18	27	42	43	63	39	45	19	95	91	73	55	43	17	54	17	20	17	45	48	14	41	47	55	37	39	26	38	33	84	97	42	42	94	52	57	53	13	27	51	18	68	19	70	27	89	70	15	58	45	10	79	42	98	27	42	36	94	46	70	85	76	30	81	70	48	10	50	87	20	43	81	18	22	90	84	50	34	76	53	13	62	12	84	80	39	34	75	49	25	44	89	45	57	16	40	32	91	84	13	60	71	56	74	36	61	99	11	68	97	39	24	68	59	58	26	33	44	22	34	40	34	17	29	77	79	63	52	51	24	26	33	93	43	61	45	39	45	80	99	78	36	26	69	55	56	81	70	89	61	59	58	13	48	83	19	38	36	93	63	75	77	14	47	83	31	55
71	50	82	90	52	98	15	66	37	13	76	25	68	98	23	60	86	47	21	52	59	74	36	98	99	54	64	88	80	60	57	20	46	16	71	92	87	89	67	33	68	59	55	40	60	86	85	63	61	29	64	65	85	35	56	50	40	60	55	75	21	77	13	65	25	91	17	60	84	76	47	15	31	19	40	71	19	91	38	43	86	52	50	70	15	67	69	97	60	51	60	86	95	60	59	31	47	19	92	96	86	87	37	65	40	17	38	15	87	93	93	24	64	42	87	78	60	26	40	95	86	25	23	68	14	40	83	66	73	85	67	91	21	74	31	26	45	41	32	37	12	56	89	63	45	80	22	71	45	30	42	65	84	88	31	77	17	15	42	44	51	27	73	21	38	59	36	33	49	92	43	66	80	79	76	77	29	76	55	17	11	70	13	55	76	71	15	24	16	95	59	79	23	90	31	63	32	21	38	86	30	75	13	20	48	47	85	12	27	51	72	18	40	85	21	24	89	16	58	85	81	86	91	66	32	72	89	86	92	75	21	39	42	93	98	61	41	71	11	21	46	31	24	91	99	93	99	20	92	22	76	95	67	19	16	78	18	42	66	77	48	54	85	67	22	87	85	69	92	42	33	18	57	71	70	58	70	66	58	49	83	50	35	12	45	70	18	69	94	96	52	65	12	45	10	20	63	59	47	28	61	78	74	62	81	44	58	30	66	78	62	23	84	65	68	39	80	10	43	73	85	64	26	36	97	96	35	22	45	27	58	93	70	12	11	90	97	28	67	31	29	58	80	95	84	44	61	49	35	43	47	96	93	26	53	96	48	70	96	98	85	66	37	13	84	51	34	92	52	50	90	70	73	31	43	90	88	26	17	46	76	68	72	54	57	57	61	27	25	44	31	16	17	64	16	80	51	32	55	83	40	62	40	47	13	16	12	67	81	52	66	73	52	91	55	29	32	88	59	52	91	75	26	95	17	77	76	62	68	41	88	19	77	26	25	96	13	76	79	76	69	43	59	12	43	53	28	55	70	57	26	22	94	78	72	47	73	41	86	16	61	56	37	59	37	34	51	74	85	43	78	88	22	63	69	30	47	93	48	92	22	39	86	43	23	42	37	55	16	11	78	13	58	12	56	13	62	50	74	35
26	33	84	30	86	58	17	14	55	91	64	73	69	80	37	55	69	99	64	48	70	96	28	32	67	76	43	50	54	67	72	38	16	73	49	96	85	47	68	27	57	46	33	77	43	37	11	40	61	27	54	33	87	97	89	16	51	81	53	66	23	98	68	39	98	96	88	15	95	50	59	47	22	69	97	45	82	80	82	27	58	44	44	20	11	49	41	25	77	26	85	44	76	61	83	54	10	76	24	77	53	64	79	51	41	49	99	60	86	95	86	80	33	19	41	67	95	64	42	39	80	47	85	96	54	20	16	10	49	20	96	49	87	30	17	72	79	64	79	77	84	97	74	39	95	20	29	92	30	62	55	60	27	37	85	65	93	97	85	47	71	85	66	57	35	84	21	73	45	16	83	89	76	47	95	77	24	74	98	59	10	52	40	40	41	32	64	78	76	80	47	48	40	10	72	13	17	76	79	33	71	26	40	86	84	11	58	28	46	93	18	53	10	37	16	91	48	47	70	88	83	18	38	26	65	49	54	57	100	98	72	54	12	99	69	49	20	26	93	95	44	29	26	64	30	91	17	22	79	70	22	35	94	60	90	46	15	96	51	27	98	92	65	55	57	72	81	23	57	77	56	87	38	85	64	52	81	23	55	18	91	87	81	35	57	61	39	89	67	24	59	67	91	22	74	14	19	40	92	78	51	90	56	76	69	39	18	56	29	28	79	76	28	36	26	29	36	53	77	76	69	93	78	48	55	19	63	28	37	21	23	49	84	86	17	32	85	41	97	46	44	62	43	77	87	70	42	60	73	40	84	96	100	12	73	26	59	68	45	71	45	39	86	71	32	27	11	55	76	82	11	35	34	20	33	67	43	42	22	77	57	16	61	36	48	97	96	47	43	24	37	99	77	69	83	27	67	15	44	98	31	94	43	67	34	70	86	36	49	47	87	68	15	19	44	91	99	30	62	66	93	48	68	59	60	71	67	23	87	52	50	76	82	30	97	68	71	11	27	35	19	30	38	33	75	30	81	79	66	86	49	12	78	98	70	48	80	93	17	70	14	51	19	79	63	27	88	31	72	53	65	78	81	79	50	65	14	54	62	66	18	50	56	89	95	21	30	76	17	85	51	81	67	80	72	22	15	43	66	70
47	76	44	67	39	94	76	90	33	20	82	14	88	70	38	14	60	60	78	75	24	55	80	30	89	26	28	70	83	86	28	56	89	26	20	54	22	27	97	96	20	18	39	70	83	82	42	38	36	60	47	98	50	14	29	72	50	74	61	49	79	50	97	41	24	75	13	62	77	67	95	70	22	65	63	73	32	13	34	91	55	85	90	47	47	77	91	11	97	30	60	78	37	56	50	20	66	34	89	56	90	74	18	46	59	77	46	74	56	60	46	67	21	39	25	61	95	56	76	43	67	48	72	86	36	62	39	83	15	26	16	87	48	60	29	100	60	16	40	87	84	82	74	84	54	12	33	44	31	15	73	59	92	56	28	71	95	62	16	37	12	53	93	71	34	86	63	44	45	59	93	20	85	34	24	15	81	84	35	64	90	82	73	22	42	60	49	77	17	21	66	20	24	45	22	51	89	97	21	91	46	18	81	81	54	12	90	60	43	19	53	93	14	76	72	99	25	32	22	82	22	72	56	98	71	70	54	16	14	50	38	98	69	22	47	45	72	42	14	25	88	38	29	33	94	49	11	54	12	14	69	14	25	83	40	15	10	92	75	64	57	35	42	43	50	79	39	60	33	24	30	66	93	89	33	63	48	73	34	66	30	53	42	83	72	68	80	53	18	54	72	21	38	67	73	86	82	34	61	50	53	63	66	40	16	27	94	44	97	69	99	91	65	88	58	57	98	47	59	21	71	69	39	17	11	69	16	65	53	25	41	44	28	47	61	42	61	72	30	41	26	55	11	74	50	52	78	75	27	15	66	51	16	24	61	57	33	94	13	19	76	84	95	34	73	28	59	21	44	70	69	23	35	21	59	90	91	60	95	26	49	68	84	45	73	36	25	29	70	23	22	56	81	21	66	58	95	27	30	62	88	76	82	78	96	29	74	57	100	44	77	15	78	73	53	75	35	96	92	44	53	76	77	17	38	94	34	11	62	88	70	55	31	13	29	31	95	30	15	96	24	82	72	16	80	84	63	44	23	20	81	18	22	37	53	84	70	21	95	64	51	69	57	99	34	84	83	16	91	58	56	72	23	83	80	16	14	75	74	78	36	76	13	26	44	30	60	67	71	80	68	66	45	30	95	92	75	61	53	23
46	75	63	50	91	47	67	12	31	29	81	81	81	90	46	25	34	59	30	14	94	91	53	89	41	96	77	71	32	70	91	78	74	19	78	61	98	80	74	37	49	47	29	35	82	84	53	25	32	50	50	62	43	40	43	13	88	46	23	83	11	90	39	71	24	65	96	43	49	26	76	44	33	21	86	52	14	70	87	89	20	20	38	48	59	16	91	43	32	49	85	14	75	99	38	62	84	57	71	36	21	19	23	45	98	69	27	40	88	83	31	56	98	87	69	43	83	27	95	80	53	39	51	81	35	93	72	67	45	30	73	20	57	71	99	21	53	47	86	88	81	31	36	32	24	43	55	94	31	61	80	17	88	38	73	56	21	95	79	39	46	60	63	46	31	28	45	61	12	19	22	24	58	45	19	36	55	47	25	86	54	90	99	72	94	67	64	44	16	36	51	37	16	17	89	16	80	33	60	68	30	83	94	25	86	20	75	22	17	53	14	52	35	91	25	89	73	68	37	99	35	48	83	62	57	90	68	60	70	89	67	25	25	50	62	83	13	18	68	27	60	76	15	88	87	20	18	54	99	48	26	51	57	73	29	99	67	31	43	34	61	11	70	33	99	89	26	28	22	92	89	25	15	99	55	49	77	68	93	66	68	55	76	13	71	27	60	42	99	47	79	95	31	68	63	60	34	70	85	89	33	38	67	26	69	80	25	64	97	88	94	43	15	28	22	29	25	50	20	24	24	90	62	44	61	64	78	43	87	84	63	99	75	32	49	53	61	65	84	13	37	86	98	65	47	44	43	40	73	86	80	80	80	32	57	40	40	61	68	99	19	32	100	71	87	73	73	39	33	72	79	84	57	49	52	23	30	20	83	34	34	39	30	68	28	23	81	62	21	22	47	67	64	94	14	51	46	32	40	25	48	44	32	44	69	25	90	43	62	23	54	90	50	69	25	35	92	65	38	35	72	26	49	81	41	97	90	63	38	63	21	67	37	85	25	12	40	30	90	30	58	55	53	43	94	83	99	21	64	27	89	42	12	40	58	39	47	11	96	94	13	93	71	16	50	91	44	18	99	63	98	39	60	43	52	41	99	50	30	20	66	91	65	88	69	77	75	63	52	70	54	31	67	66	55	19	21	59	95	25
94	56	23	66	35	92	25	24	93	86	25	28	51	49	17	27	32	50	36	23	79	98	87	81	44	68	57	73	94	22	50	99	82	95	68	12	100	95	95	98	64	22	95	35	76	85	49	95	70	10	62	17	63	94	66	12	78	49	95	45	27	19	39	58	24	54	79	76	26	90	34	99	14	43	38	85	11	21	44	59	90	43	58	72	61	64	12	24	64	39	15	100	50	68	40	50	26	64	88	97	74	13	18	12	18	32	25	44	58	21	51	72	48	17	47	67	24	81	58	82	44	86	71	14	35	74	29	87	63	84	88	49	79	76	91	44	44	15	74	32	94	91	58	32	31	51	14	89	67	93	39	100	51	44	85	52	68	16	49	75	72	38	19	31	29	34	84	13	82	27	85	78	74	53	57	79	40	56	48	39	52	42	85	56	96	57	43	66	37	47	41	29	27	23	89	64	59	98	24	83	82	18	50	79	76	65	86	75	43	12	55	25	34	40	41	60	44	14	61	50	69	57	11	42	46	79	28	38	39	94	12	55	95	73	92	95	87	37	34	15	86	71	77	32	31	87	29	92	36	20	86	41	48	48	83	64	71	23	26	34	29	53	95	72	58	56	78	67	85	14	55	30	80	63	40	51	15	31	35	12	12	30	64	42	52	53	94	39	90	70	40	91	19	69	56	75	39	38	84	90	43	78	39	71	88	86	37	39	23	16	87	72	62	100	79	37	84	89	47	53	54	33	97	76	98	47	23	72	70	86	35	68	70	97	77	89	15	31	62	32	35	68	60	84	16	93	91	17	14	78	58	61	29	87	76	16	70	64	25	95	39	10	18	24	80	40	12	11	87	59	15	49	49	48	41	87	57	46	11	30	84	53	56	72	10	64	78	98	59	71	71	89	14	54	76	88	78	96	77	83	31	43	49	50	12	71	80	61	68	65	64	62	69	45	52	76	11	40	87	52	91	87	20	74	88	61	95	14	16	85	15	52	64	88	87	19	19	81	15	50	99	47	65	58	94	83	94	46	67	86	20	70	12	79	62	57	38	23	62	36	90	14	45	42	29	42	96	62	26	25	64	63	28	26	22	31	43	15	26	81	64	17	11	69	61	64	21	49	61	59	54	81	99	53	85	41	44	36	21	93
90	68	91	18	43	38	70	79	23	67	33	52	14	92	37	49	93	21	64	51	88	23	23	30	47	91	99	60	12	18	82	75	15	29	32	46	97	79	97	26	83	26	44	55	64	81	35	11	28	72	30	87	37	91	39	65	16	76	46	26	21	85	46	26	44	37	34	46	73	80	21	98	78	86	45	64	15	88	57	55	48	62	57	41	25	91	68	81	83	37	50	43	49	58	82	83	48	69	70	37	70	47	53	11	82	66	26	11	66	15	97	89	52	18	83	86	51	69	59	76	70	24	17	79	23	88	99	37	86	17	89	13	98	40	97	16	92	27	41	71	86	95	22	18	97	16	83	99	60	80	27	17	38	56	76	44	23	75	43	33	92	51	74	86	77	81	41	31	64	96	17	88	92	51	57	29	47	34	83	65	45	98	56	35	21	11	61	73	77	24	38	14	15	32	47	58	93	89	28	82	94	27	62	13	82	86	44	17	96	11	85	50	81	45	62	90	71	64	82	37	100	22	79	82	90	77	68	18	43	36	67	99	33	69	79	46	21	92	75	33	92	99	31	62	11	98	65	29	72	98	32	16	35	10	96	35	31	40	76	93	86	24	37	72	56	35	58	80	42	84	77	75	100	28	14	96	18	96	67	82	66	15	54	50	47	52	17	32	33	76	54	43	48	31	13	36	24	18	16	94	86	11	58	83	84	43	34	59	78	12	53	53	16	76	32	80	80	95	94	66	18	77	57	39	57	23	60	34	26	41	86	22	83	20	60	92	17	63	39	61	86	17	32	44	82	65	74	79	78	83	39	52	29	83	86	90	26	92	67	23	87	65	39	41	82	67	99	45	34	44	72	44	53	72	84	28	70	25	75	51	55	83	48	66	84	43	74	70	44	47	87	73	29	83	16	25	77	88	31	15	64	74	63	76	82	31	100	93	33	97	12	17	48	37	63	47	77	16	97	92	86	47	84	50	37	39	17	29	20	68	77	13	17	54	32	71	37	85	38	66	17	37	42	76	81	83	69	81	52	13	11	50	66	97	35	15	96	80	59	96	30	11	50	52	35	83	72	61	61	43	57	53	93	26	98	59	79	10	74	31	35	74	84	39	65	78	24	83	67	58	93	92	32	79	49	97	24	85	72	15
30	61	65	38	93	97	53	49	81	44	87	12	98	14	27	44	86	75	58	20	80	73	14	99	88	23	69	47	34	70	88	39	43	39	45	59	10	100	79	52	84	99	43	78	61	22	74	13	97	61	22	64	76	95	37	72	87	23	32	63	26	95	22	94	82	35	83	92	64	83	36	33	64	40	68	53	13	89	27	96	23	36	87	80	83	29	49	69	82	27	60	23	33	84	71	89	47	56	53	47	60	96	21	82	55	75	42	59	66	64	59	92	26	69	55	42	99	59	27	72	35	69	73	13	89	13	28	47	39	13	87	75	75	28	99	52	54	83	68	16	34	62	42	14	98	72	94	29	41	13	44	79	40	38	86	19	61	87	16	64	65	28	90	18	14	77	28	99	48	90	30	40	74	22	80	85	64	43	52	41	63	19	23	63	79	64	29	72	85	51	91	17	71	83	72	39	18	79	74	30	19	76	25	99	61	66	13	13	31	22	62	48	93	66	22	42	27	50	80	56	38	64	98	57	71	56	59	75	85	39	89	47	23	39	37	14	55	65	83	96	75	34	41	12	95	84	23	37	15	82	42	78	20	28	58	62	39	77	14	76	11	62	39	29	99	21	66	95	31	64	90	19	26	45	77	53	98	36	60	91	39	50	14	13	51	35	38	57	26	43	70	10	99	38	80	30	65	41	34	40	89	65	41	73	37	14	35	48	85	36	86	94	62	86	99	67	53	78	11	93	32	71	97	43	53	45	32	34	86	12	59	28	70	44	30	87	60	74	59	83	70	87	46	57	46	62	18	86	82	37	51	89	99	63	86	98	100	32	27	76	13	54	21	78	68	41	81	74	80	15	18	86	54	15	74	73	70	12	23	37	22	46	84	81	68	85	16	95	33	99	39	27	53	19	76	74	35	48	77	48	67	19	55	46	78	40	87	68	14	26	42	99	17	45	61	50	59	44	13	20	59	42	24	48	22	26	48	51	99	93	52	93	49	36	21	34	75	96	22	60	13	94	71	43	75	31	46	60	60	66	80	53	54	80	77	83	37	92	85	88	84	84	70	55	14	63	35	72	19	14	85	32	42	19	88	67	51	31	88	42	32	97	53	79	32	14	15	87	77	30	21	82	20	74	26	20	72	45	75	82
58	74	33	79	45	48	85	86	21	67	58	48	40	38	67	23	56	68	84	69	58	66	66	12	12	39	13	99	15	46	55	52	44	96	27	58	15	14	57	29	93	57	44	22	70	36	71	91	78	79	87	21	19	70	90	41	52	76	66	66	15	92	74	21	58	54	64	63	31	82	79	99	47	66	71	95	68	36	32	92	26	57	51	82	55	45	92	32	42	57	18	72	21	55	78	81	76	91	52	56	53	95	41	23	57	96	55	32	78	97	76	30	49	88	74	27	64	43	41	45	79	23	51	97	17	92	79	45	69	31	54	90	57	56	25	31	73	31	31	90	23	47	17	34	94	60	30	33	12	28	78	16	91	54	91	45	40	94	31	71	39	83	37	58	23	20	59	89	25	90	42	72	42	26	48	17	19	42	99	100	56	42	26	70	98	74	44	91	49	30	56	74	99	17	51	26	77	81	34	80	68	59	40	43	15	95	27	45	52	67	85	60	59	69	51	53	69	69	52	41	48	88	22	73	42	58	91	75	64	82	17	76	73	94	53	19	25	46	44	20	22	94	52	49	39	62	72	25	23	62	21	18	46	36	46	12	13	15	64	56	26	79	52	34	38	45	75	16	40	92	77	37	47	36	53	80	39	43	94	56	93	97	25	71	88	39	23	36	46	15	98	97	24	63	79	65	68	35	86	43	44	42	66	97	44	93	46	80	59	45	39	41	26	77	70	15	46	34	92	36	54	10	52	61	17	16	40	65	97	67	38	19	52	95	43	58	86	91	70	97	41	62	35	72	61	84	17	20	31	43	88	91	41	50	33	91	46	79	88	89	79	77	39	90	84	22	56	89	50	82	53	13	62	45	33	28	52	50	56	64	31	28	95	62	44	39	17	68	89	52	79	50	12	87	96	69	11	48	56	18	61	10	22	91	77	92	42	66	75	78	93	47	30	30	96	45	46	23	91	98	11	75	60	17	69	51	16	30	54	29	20	13	85	59	60	38	64	71	33	41	40	54	21	73	62	66	87	39	52	44	54	58	82	70	95	61	39	70	96	31	50	58	87	23	83	25	52	42	75	77	14	62	44	37	63	52	93	94	17	45	28	47	29	12	50	76	49	80	70	53	48	35	36	43	58	25	93	99	55	45
52	59	53	76	11	81	26	75	66	10	67	62	62	22	97	30	50	92	90	58	51	97	49	11	14	68	25	15	51	13	99	49	23	32	67	54	17	93	99	85	48	27	43	85	78	45	46	64	68	86	29	63	91	66	55	26	21	80	37	12	30	54	53	30	21	30	12	95	34	65	42	30	30	71	18	14	94	85	22	43	79	44	95	84	27	22	46	45	46	85	55	23	16	62	80	52	99	72	43	36	53	15	55	53	41	98	86	53	61	83	46	93	37	89	57	86	64	30	55	54	53	83	71	22	22	16	21	85	82	24	88	13	96	67	15	25	52	24	34	80	28	96	41	18	35	20	56	20	42	67	74	35	34	47	99	20	19	47	42	63	68	85	96	31	77	12	75	61	76	35	26	99	10	47	97	30	83	53	27	58	29	23	67	85	60	57	96	85	36	96	45	50	43	98	69	25	89	52	61	29	69	26	54	28	34	85	72	26	79	91	31	13	77	28	78	38	46	19	83	93	63	76	98	75	34	63	44	67	14	52	75	11	63	64	82	45	75	44	24	45	40	45	48	36	91	57	71	97	69	90	77	41	13	29	100	36	57	50	14	15	90	23	50	40	11	31	60	74	37	70	69	93	87	93	98	16	23	72	88	87	69	48	77	67	54	78	46	80	97	72	58	77	72	77	77	54	30	26	30	21	41	79	27	63	44	36	87	28	48	58	80	80	88	27	62	72	60	28	87	70	65	74	64	59	32	93	80	32	90	74	87	83	75	44	47	30	54	66	30	22	62	51	93	25	11	52	17	86	44	80	59	68	49	68	74	35	19	26	18	35	97	17	52	60	96	60	35	32	83	43	10	19	89	32	30	52	68	68	64	68	66	66	15	79	63	79	87	27	59	83	24	59	70	88	20	13	85	79	80	24	48	100	44	30	26	61	18	82	69	70	57	27	52	79	38	65	95	28	51	17	83	20	91	68	46	28	18	63	90	82	30	41	51	74	38	23	89	86	84	49	52	44	96	80	98	97	22	56	22	44	93	21	36	20	66	79	25	18	96	37	83	39	95	99	31	60	77	42	54	53	32	49	23	71	34	12	97	84	92	83	21	37	43	97	68	68	59	33	27	98	42	33	38	46	46	50	12	65	75	46
70	18	53	77	65	84	97	27	92	18	71	68	19	67	39	58	69	40	60	14	57	81	82	43	76	93	20	54	57	34	53	80	66	58	37	44	26	20	95	22	40	98	80	95	31	23	66	58	35	25	43	96	94	77	90	10	15	55	59	50	75	37	85	64	62	84	72	87	29	95	45	29	83	14	71	72	74	56	97	91	64	10	14	70	69	59	18	36	30	48	12	49	51	97	85	52	59	27	83	50	79	99	94	95	96	16	76	36	59	68	16	87	18	97	68	57	27	51	52	24	71	35	41	37	45	84	90	26	71	39	61	65	32	35	64	51	53	52	48	31	87	26	63	19	68	75	47	75	48	96	95	89	69	86	91	87	72	25	52	72	67	58	66	22	54	94	64	53	11	97	27	64	49	76	79	61	54	28	25	68	62	20	20	31	35	93	31	29	55	44	59	93	21	14	92	18	27	69	32	28	92	14	80	88	10	55	73	57	26	34	16	58	40	99	94	99	12	41	45	84	27	93	46	23	41	73	74	66	41	81	38	58	85	94	48	53	90	98	98	65	88	78	53	27	10	11	36	74	70	90	70	42	88	88	93	100	90	38	68	84	53	16	91	24	83	79	47	57	35	64	95	89	79	88	36	89	20	10	38	99	45	97	97	43	55	47	90	81	14	87	14	61	96	88	14	95	43	19	61	71	52	77	73	47	64	35	35	64	94	23	57	32	11	74	28	34	72	27	88	16	60	38	23	31	10	15	78	58	30	47	21	93	88	97	65	40	36	96	59	34	81	56	31	24	11	30	82	48	63	10	23	45	29	94	54	93	53	51	49	65	85	93	27	85	35	64	85	51	28	55	98	35	79	74	20	100	97	10	59	69	85	81	99	85	33	82	30	64	44	48	18	46	40	38	24	23	37	92	67	17	64	11	47	86	77	85	82	15	25	63	84	22	60	34	56	13	77	25	46	28	77	32	97	32	73	32	95	98	75	47	22	19	38	66	22	44	93	38	89	65	65	72	74	32	22	94	98	38	39	93	17	83	12	61	22	31	18	56	50	34	69	44	69	93	98	64	28	41	54	13	57	23	61	25	33	31	75	37	95	29	75	13	13	43	60	18	96	62	54	86	61	57	19	77	73	79	43	28	16	94
37	53	30	22	89	37	86	74	34	62	80	58	77	23	37	94	82	11	54	37	83	28	67	34	62	31	52	30	65	68	81	89	70	24	10	14	18	20	20	31	36	73	97	65	82	45	73	23	53	23	31	80	63	13	96	81	76	99	53	44	38	96	61	27	92	86	28	62	66	65	41	89	36	85	82	77	33	88	31	73	52	99	71	89	10	81	47	40	29	97	45	17	97	26	58	62	84	20	78	96	22	90	90	23	35	77	15	13	65	13	37	42	36	66	27	60	60	13	16	61	92	22	22	95	35	65	24	97	38	66	90	13	30	94	49	57	78	98	37	74	67	76	52	22	54	38	89	37	59	27	60	27	85	31	76	14	91	97	46	60	51	55	58	95	21	82	72	44	46	20	85	21	46	77	85	45	62	65	98	100	67	99	62	41	79	76	38	90	90	39	27	98	60	45	28	75	93	68	100	54	82	89	68	85	32	42	73	59	80	28	39	13	88	24	57	96	32	81	22	84	97	92	17	75	92	58	89	15	20	94	81	58	19	11	87	42	20	56	30	78	58	20	50	79	86	40	56	23	19	16	99	88	56	60	74	55	97	57	85	59	65	60	64	13	22	30	17	46	48	94	99	54	49	80	45	15	87	20	71	42	57	22	22	24	50	66	22	76	20	51	76	54	76	14	88	59	81	35	62	26	22	91	49	42	57	84	11	69	68	81	14	13	67	30	96	24	97	23	85	92	32	76	33	57	14	21	51	22	51	59	83	12	45	30	12	12	66	68	45	63	94	43	64	21	82	68	65	71	81	56	33	73	77	92	20	28	72	82	77	28	61	21	23	48	71	86	97	44	52	57	15	32	37	41	83	65	37	49	11	70	41	80	83	71	27	60	16	12	93	37	68	94	59	59	54	79	33	27	52	44	74	33	53	97	85	45	100	80	46	61	94	17	24	56	41	98	41	99	14	40	80	74	83	17	80	16	54	68	93	49	68	56	40	21	42	77	30	60	24	71	35	49	76	75	15	46	13	71	50	72	25	32	61	48	49	28	75	83	13	17	84	91	97	92	26	25	41	79	77	70	35	46	29	62	58	36	23	42	59	87	45	87	83	76	95	55	54	57	24	63	64	28	39	60	65	32	34	84	26	24
95	50	26	67	52	71	61	85	94	21	79	68	28	13	22	27	62	78	53	40	47	83	61	73	67	60	19	47	43	98	30	20	87	42	92	38	97	74	20	84	15	32	68	94	24	86	92	73	85	15	22	18	74	84	98	38	39	24	53	13	78	54	77	36	18	35	81	69	82	46	33	31	79	79	54	63	48	61	18	29	75	23	60	70	58	12	47	87	71	61	74	69	21	63	66	96	86	84	72	12	94	33	18	61	18	35	18	98	41	38	63	11	14	85	86	72	35	11	67	69	47	84	92	53	88	24	79	80	45	73	66	80	44	36	95	74	51	69	94	57	36	27	47	44	83	54	55	30	59	66	85	55	28	35	85	59	22	97	66	87	46	27	31	55	68	97	25	46	45	54	85	67	28	92	11	71	32	56	24	49	16	30	82	56	54	86	91	17	48	86	22	22	79	60	79	83	43	60	14	12	54	23	29	72	75	18	89	75	37	55	15	73	81	66	87	15	12	96	92	21	50	70	29	88	64	38	52	36	74	60	56	66	58	36	46	84	27	20	45	51	96	21	22	65	39	54	75	67	61	85	39	77	63	18	35	31	42	23	65	42	99	61	37	75	21	34	48	45	55	23	99	90	88	86	67	44	61	74	53	82	24	88	41	55	47	78	72	28	32	50	87	45	80	92	70	84	100	30	24	66	23	71	69	85	32	75	100	88	56	40	93	51	90	100	35	100	77	94	32	78	97	93	30	23	35	11	43	99	57	16	97	29	70	83	76	54	23	39	42	99	92	74	24	34	77	14	25	74	98	20	48	99	76	87	89	34	64	68	46	93	18	94	86	64	85	68	99	51	17	36	70	91	59	11	34	16	34	98	31	94	76	27	86	50	40	84	74	85	25	28	21	31	14	73	20	98	84	76	91	88	54	62	11	14	68	86	15	30	35	24	88	91	18	97	75	80	61	19	78	13	77	30	70	28	16	59	19	63	46	97	39	48	10	52	37	37	49	28	44	14	67	47	12	45	85	36	66	15	78	68	52	38	85	27	58	25	20	67	41	78	14	85	45	22	94	19	46	42	89	19	84	48	77	34	62	43	68	95	57	31	72	37	87	54	17	33	75	29	64	99	36	35	78	78	13	13	82	25	21	37
74	37	96	100	42	36	22	11	87	91	86	35	68	54	50	68	36	32	24	99	18	14	81	80	24	52	27	83	21	66	52	78	88	82	38	28	31	54	44	20	19	100	69	45	60	75	41	99	83	65	86	11	89	88	38	84	32	22	95	78	93	41	14	61	61	49	27	11	36	52	73	61	88	68	11	98	72	16	73	46	24	90	83	68	98	51	63	72	28	69	50	65	79	89	23	24	48	87	50	33	18	98	38	25	33	92	31	37	38	36	21	100	83	31	18	67	16	74	31	61	20	77	93	95	48	99	70	85	41	78	98	77	93	53	25	64	77	98	24	18	85	88	45	41	67	92	91	76	79	35	50	33	24	99	15	90	24	89	78	98	94	51	32	85	76	41	84	28	71	47	46	71	63	74	56	44	25	19	85	96	15	81	41	51	50	17	13	85	17	15	88	93	27	65	90	79	44	100	42	72	51	97	87	51	21	74	78	80	65	41	53	71	71	32	31	23	93	84	84	16	21	38	50	73	39	79	16	55	77	69	25	20	52	69	89	39	54	42	33	84	73	52	98	14	51	69	87	31	59	31	39	27	61	18	46	62	77	51	12	94	56	67	85	99	53	30	58	87	68	16	16	48	74	37	96	18	97	67	88	90	55	96	44	52	53	56	96	90	14	95	43	13	43	59	15	80	33	63	84	15	42	15	66	68	14	33	63	48	33	14	83	46	22	74	87	68	99	36	69	12	77	34	22	30	75	61	25	17	79	99	35	87	31	73	10	79	43	11	99	68	47	71	65	81	87	73	73	73	70	54	82	65	45	19	97	16	94	27	82	57	79	89	84	27	98	90	52	41	33	79	87	18	93	73	74	92	47	88	34	66	92	67	66	90	57	87	33	70	65	14	16	19	67	56	50	66	24	29	53	21	34	46	68	78	15	96	24	26	57	40	10	20	77	17	61	68	29	70	87	26	24	55	74	58	84	46	73	20	80	23	41	34	26	25	21	13	80	56	85	48	96	80	11	46	30	62	30	52	96	43	17	11	50	99	81	81	48	86	62	31	84	29	39	92	15	74	95	34	81	22	98	59	94	19	33	56	97	36	61	84	77	86	89	88	38	59	22	81	37	59	66	59	33	41	81	94	70	75	13	22
99	84	67	39	97	77	27	14	50	66	23	92	23	92	60	97	38	17	90	15	65	38	25	57	25	11	59	51	38	42	17	23	62	72	73	46	47	31	81	40	15	19	32	28	97	24	94	39	15	65	99	82	84	33	84	58	92	78	48	28	19	67	25	88	99	78	17	64	49	52	78	16	46	83	20	12	17	26	46	81	79	36	50	65	29	53	77	91	99	76	90	83	25	80	71	75	87	14	88	11	91	77	67	72	74	26	78	42	39	71	40	25	49	35	75	73	58	80	56	71	31	50	53	55	19	60	80	74	12	34	55	91	93	72	73	83	93	86	88	20	47	20	53	32	14	19	32	59	12	23	95	41	87	83	77	92	88	26	82	63	56	98	89	83	20	99	12	59	21	40	65	83	43	81	88	51	45	25	13	65	82	72	91	34	35	33	10	49	89	21	17	19	56	64	11	97	85	68	89	61	16	52	36	59	53	55	30	56	73	20	99	12	56	98	51	18	43	61	56	68	62	39	96	64	93	31	14	53	53	81	50	54	47	100	39	70	13	53	30	72	45	72	26	11	77	58	76	70	46	76	23	85	63	20	46	41	52	83	25	49	97	43	72	10	43	50	35	85	81	68	86	31	85	77	55	22	51	57	96	68	57	79	22	20	76	75	80	29	34	40	80	88	16	92	87	30	23	62	42	34	88	31	72	34	34	49	94	46	67	15	24	18	85	21	35	19	32	40	40	70	36	70	57	95	40	37	45	34	77	70	92	49	89	87	52	98	72	51	53	60	20	84	72	47	25	79	21	85	15	27	84	51	42	43	98	68	96	60	19	54	80	43	16	36	28	42	84	29	77	24	53	79	34	91	50	79	87	82	83	25	84	80	65	56	55	52	82	97	58	43	10	91	41	60	71	92	38	15	64	34	81	58	32	41	95	80	77	82	37	48	89	69	40	82	80	37	22	34	39	96	62	29	19	51	43	95	45	73	11	26	20	25	64	50	96	26	86	99	69	49	87	46	91	43	92	64	68	60	21	27	65	67	48	80	46	37	88	95	92	24	55	73	54	98	85	35	92	52	91	22	51	23	15	42	39	58	77	14	62	66	65	54	28	18	42	23	64	81	26	34	44	56	84	94	47	84	21	55	64	51
82	79	36	30	39	46	98	56	79	60	74	69	42	70	87	57	12	82	86	21	62	56	78	84	97	89	24	22	72	13	74	13	63	66	83	21	13	89	94	98	42	48	67	55	18	89	17	72	38	98	18	44	20	18	77	25	85	65	54	53	53	68	63	57	72	15	26	68	73	88	81	44	30	56	22	10	33	78	70	53	31	39	99	16	12	41	88	15	73	78	66	77	22	94	38	10	18	38	66	42	29	24	92	95	53	29	96	68	51	16	78	73	44	75	35	13	78	26	29	98	18	21	62	79	58	71	90	88	46	78	80	98	64	52	75	42	14	69	86	62	65	77	88	74	97	80	21	62	38	60	62	68	11	75	64	33	20	11	79	39	88	96	95	45	65	73	54	42	95	27	82	37	47	99	31	48	66	63	14	20	82	91	18	43	41	97	32	97	25	42	73	39	52	58	39	52	51	61	86	52	56	51	47	34	47	59	87	48	45	24	18	47	50	12	33	78	70	58	28	84	93	15	33	51	42	16	25	31	25	58	91	74	22	85	54	86	37	92	65	66	19	53	96	28	26	20	16	91	29	14	62	52	24	63	88	33	30	22	79	36	22	41	69	74	33	64	49	33	46	44	65	85	62	81	44	48	40	11	72	73	76	85	86	75	12	19	81	11	81	80	53	90	46	73	96	83	50	24	11	64	79	90	81	47	74	54	61	43	20	47	91	12	28	74	27	54	19	50	46	69	33	39	25	21	14	61	18	59	30	93	67	85	99	55	62	48	62	35	85	16	94	95	25	12	62	85	79	68	35	69	39	68	27	91	23	68	39	20	45	31	61	25	72	30	41	75	20	90	35	69	19	20	30	44	16	15	95	32	38	72	92	13	79	22	11	80	50	34	94	12	75	91	82	91	59	55	24	88	25	54	38	71	68	93	61	22	53	72	54	89	84	64	99	69	25	13	31	96	24	31	41	46	15	46	31	62	65	79	57	68	63	66	53	58	65	73	77	64	50	33	17	47	12	96	50	47	45	80	62	75	96	42	45	86	66	75	40	89	26	33	68	84	55	69	43	39	98	99	55	32	62	50	56	100	21	57	61	79	23	28	75	54	56	52	41	82	98	47	45	28	56	76	62	78	18	83	15	27	54	72
78	85	55	71	37	57	27	13	76	37	34	81	19	17	80	25	69	33	91	79	42	79	20	23	13	26	94	45	84	32	67	68	32	70	99	30	17	21	34	38	37	66	91	88	79	64	25	28	48	37	23	43	11	40	61	31	37	43	12	41	86	13	56	48	83	42	80	72	83	63	31	37	26	48	99	100	56	76	21	69	46	49	73	83	30	13	50	52	26	99	35	85	74	38	67	72	35	64	99	86	28	34	59	55	100	81	12	96	68	84	76	69	31	65	41	99	96	48	52	37	99	84	48	54	63	34	77	37	17	98	67	54	24	60	52	77	41	43	67	64	26	83	75	62	22	12	55	18	95	96	55	71	78	75	91	66	93	59	31	30	58	76	17	77	17	23	70	14	12	76	71	50	60	83	40	71	58	52	28	48	56	13	63	79	70	16	49	88	92	16	42	97	22	54	77	68	58	70	42	29	72	29	58	44	64	92	61	83	86	29	78	17	72	39	97	12	72	93	89	99	46	59	12	78	65	30	56	60	70	44	41	38	32	76	18	39	93	51	68	27	48	35	49	25	88	83	34	33	92	42	95	35	38	37	37	21	31	32	85	49	29	11	46	79	89	38	36	44	83	34	35	13	95	36	33	88	71	38	70	84	55	32	67	13	61	91	88	98	62	25	33	23	56	42	70	90	47	12	61	57	52	16	62	50	28	67	24	66	26	14	68	84	37	76	23	62	31	49	40	41	59	98	12	78	97	38	58	80	60	72	25	79	39	15	11	60	14	91	62	82	72	94	92	19	54	16	87	71	61	57	88	41	25	63	74	40	55	15	91	59	19	53	24	47	65	48	99	86	35	23	92	49	63	98	73	42	76	70	67	73	25	76	72	77	72	41	93	83	33	100	90	89	63	41	76	46	87	84	90	77	35	93	38	19	97	79	64	28	56	18	74	44	29	20	69	42	51	44	50	23	87	71	89	74	77	68	52	43	83	18	76	50	96	39	94	78	36	30	81	23	23	89	75	54	52	59	58	84	12	88	96	55	15	99	59	69	22	13	25	19	57	20	65	85	85	95	19	36	57	15	97	86	23	74	98	41	47	71	55	86	79	21	76	48	12	51	47	82	52	78	34	18	51	81	35	16	25	80	41	69
79	38	75	45	21	60	14	94	52	96	94	32	45	100	84	69	24	76	67	60	95	13	80	38	57	27	96	62	99	53	50	64	82	78	45	47	45	37	68	32	95	96	65	19	30	96	72	16	65	81	61	17	46	61	88	19	22	59	94	26	47	93	74	45	89	100	96	72	61	31	98	82	87	95	21	73	96	90	78	75	50	18	61	34	53	90	73	74	59	56	49	49	37	77	16	21	77	41	95	56	81	91	30	59	76	22	14	22	63	34	74	78	52	40	81	33	91	74	44	29	61	75	73	57	26	17	67	66	100	62	20	88	60	31	51	98	11	60	34	39	13	37	39	26	28	71	88	39	65	82	34	83	90	53	51	73	18	21	54	82	52	24	89	40	51	70	30	99	68	33	43	94	88	91	36	32	74	67	65	60	67	97	11	20	66	10	84	80	31	14	85	78	74	30	12	88	45	47	39	75	30	62	66	42	59	77	97	33	64	36	79	35	21	96	88	77	20	27	38	88	38	25	25	99	87	99	32	98	46	36	24	93	15	24	87	90	22	22	100	45	79	54	75	11	39	20	17	54	93	57	92	48	81	45	62	65	93	52	64	85	61	69	71	35	12	73	27	54	52	21	66	61	52	66	96	43	79	82	68	49	63	68	61	82	15	12	44	50	75	68	24	23	33	97	20	62	50	34	19	90	60	85	42	68	60	95	36	84	97	33	47	63	71	17	12	46	30	15	58	70	50	97	91	53	79	19	48	69	65	58	50	97	20	97	79	83	76	39	20	32	98	68	70	60	53	96	70	88	47	79	23	52	85	98	80	85	24	31	38	22	69	27	22	95	78	77	10	62	47	12	18	66	87	18	23	79	17	99	28	29	36	22	84	17	49	82	90	25	82	95	24	14	34	92	94	42	11	89	11	18	59	53	46	100	72	27	76	77	73	63	37	14	85	29	87	67	47	53	64	33	32	81	71	25	29	19	73	56	92	42	57	77	57	61	35	38	29	51	80	89	71	71	57	98	32	97	59	54	42	89	92	34	31	43	16	83	16	58	38	22	19	87	72	70	91	94	63	16	80	93	42	100	17	94	56	29	83	29	67	64	40	67	11	31	33	31	72	62	74	10	48	43	12	64	48	46	30	58	61	34
43	31	54	92	37	16	96	13	94	16	30	58	85	72	30	40	90	58	41	38	84	40	56	11	83	67	56	40	76	30	66	74	19	98	85	99	49	74	39	14	48	97	90	71	91	79	16	61	74	81	61	68	42	81	60	98	30	93	60	71	46	90	20	59	25	31	57	45	49	23	74	52	18	14	83	21	46	33	47	76	82	25	44	52	92	56	64	81	86	87	82	44	70	92	51	71	81	90	40	62	76	72	88	24	97	56	61	63	34	49	81	70	40	37	34	57	18	99	99	20	34	16	48	26	32	94	46	98	37	69	36	83	95	16	73	52	25	89	32	18	83	53	95	94	19	86	95	62	15	97	77	84	74	56	36	64	95	14	83	19	42	35	91	81	76	57	45	43	51	12	51	99	12	33	20	100	32	22	55	95	50	31	39	38	89	72	32	78	42	49	90	76	21	78	95	79	61	17	45	61	23	28	71	17	57	85	12	93	74	14	47	62	19	22	51	63	54	77	63	34	19	73	94	35	69	27	85	12	92	62	26	34	38	17	83	12	26	97	61	82	64	49	56	50	99	58	55	71	20	54	25	83	71	61	23	73	17	75	55	84	46	17	14	93	91	63	27	75	59	52	91	98	65	77	100	65	66	79	20	31	58	76	56	32	30	90	24	77	41	17	53	54	66	64	95	95	36	14	70	22	74	83	77	74	64	81	75	25	21	30	19	85	35	29	15	40	19	73	90	95	15	57	40	73	83	28	31	39	40	88	11	53	73	49	63	99	99	85	62	70	20	70	49	63	42	31	35	69	81	15	95	40	55	46	26	59	98	89	29	29	42	44	35	88	99	23	86	37	46	78	80	65	96	41	22	28	20	55	68	45	41	96	94	79	24	40	19	15	55	62	47	58	17	81	23	86	79	18	17	38	29	55	65	10	54	28	83	63	33	35	21	11	96	33	88	90	75	41	71	92	64	44	73	92	61	51	68	78	31	83	76	94	73	29	75	76	57	11	56	13	83	46	41	35	63	19	17	86	31	23	36	42	78	85	41	16	43	63	50	97	83	77	26	78	55	40	94	79	69	30	69	37	80	11	49	43	35	60	96	70	31	77	27	49	21	20	74	92	44	28	82	99	10	18	23	65	42	14	68	33
88	94	46	63	23	62	42	19	83	22	40	72	12	65	10	29	26	73	51	16	81	71	42	19	32	27	52	93	28	12	21	60	43	47	45	52	78	90	75	76	78	86	52	30	36	72	65	54	17	78	82	84	15	59	56	92	32	91	60	91	87	23	94	88	99	41	17	43	96	26	87	26	72	52	83	18	89	73	44	48	62	12	71	31	77	97	45	29	33	98	63	21	45	63	74	79	59	38	13	61	99	58	91	94	52	13	34	97	33	23	40	88	26	31	78	11	35	11	21	100	31	23	82	83	94	30	31	82	74	73	16	94	96	57	91	19	50	91	77	39	43	30	52	11	48	65	93	53	42	32	32	51	81	46	11	61	60	60	94	69	65	70	41	88	13	50	78	44	98	38	39	92	26	82	57	49	33	56	10	40	27	97	45	88	47	78	24	44	52	49	92	11	53	31	32	85	95	96	63	46	20	87	26	93	60	90	71	53	96	16	92	25	89	80	48	45	53	25	75	59	99	56	27	18	33	82	43	98	83	62	94	71	60	85	33	83	11	56	63	76	40	95	95	87	91	100	42	37	71	24	55	12	23	30	77	62	33	58	46	30	55	29	58	60	38	30	47	67	87	63	11	43	17	82	85	59	68	84	53	40	29	13	80	82	57	92	40	46	91	11	63	66	11	38	67	14	39	44	95	17	44	25	39	24	70	38	86	23	21	40	35	16	12	27	46	18	76	93	77	56	44	63	21	20	33	74	99	99	45	76	55	35	96	71	44	93	29	48	59	40	40	23	86	73	31	99	89	66	11	59	91	17	16	62	36	77	98	93	92	95	74	25	69	24	12	48	39	63	81	29	87	67	21	38	22	63	58	44	20	67	45	73	65	61	48	40	41	64	95	16	54	45	38	39	36	84	22	62	71	95	50	95	11	10	92	57	80	52	10	61	73	52	93	63	29	43	48	77	30	25	52	96	59	61	72	94	77	68	33	12	87	93	98	51	21	27	51	19	16	49	58	75	23	50	37	61	84	79	55	34	32	51	88	31	26	94	87	61	65	43	25	30	89	32	58	55	96	82	26	87	76	73	44	74	72	19	21	50	24	63	18	75	11	87	90	48	22	77	95	84	53	89	77	64	24	84	33	20	56	81
90	87	59	95	38	42	44	39	62	55	58	44	98	24	58	67	14	39	81	76	69	91	57	55	18	35	37	69	41	11	50	64	98	44	48	93	16	66	80	53	13	81	66	55	39	30	38	50	42	75	39	47	59	76	80	70	23	23	13	44	26	70	24	16	34	17	98	30	58	69	33	23	78	39	16	65	53	77	18	62	14	19	99	25	89	13	45	34	43	25	26	81	70	51	84	34	54	60	22	20	11	37	73	17	71	21	64	39	85	84	82	40	72	38	13	30	11	72	69	41	65	93	87	51	99	37	54	54	80	67	10	32	78	82	19	68	47	22	18	44	70	95	24	16	82	62	96	19	82	65	17	29	60	24	22	34	40	47	71	80	46	45	25	48	96	36	31	68	82	18	69	71	63	81	28	81	41	37	20	84	61	95	34	24	21	40	74	37	94	50	84	64	76	85	27	15	77	60	72	37	91	42	94	57	59	91	26	59	37	51	14	18	32	91	89	39	34	36	18	74	12	45	88	54	23	43	29	38	58	97	37	14	71	82	75	68	35	41	10	56	72	64	54	45	16	52	68	22	63	30	28	43	55	54	34	13	49	16	84	44	19	65	57	24	38	79	12	93	86	11	62	25	25	79	41	30	10	29	80	68	90	87	11	91	94	60	93	56	54	61	29	19	92	94	45	16	28	92	40	19	33	37	14	84	18	99	61	19	64	25	60	42	15	42	89	60	82	94	25	12	25	40	30	18	66	55	95	42	47	51	77	20	55	28	86	33	64	44	65	35	54	44	34	27	40	15	93	93	56	47	43	68	11	39	29	61	73	25	49	88	22	83	80	29	60	76	82	54	68	40	97	52	43	19	87	18	18	87	64	89	77	81	46	59	18	29	30	44	66	89	55	26	60	71	19	45	28	83	54	61	49	18	74	56	54	32	12	60	67	29	88	65	79	97	26	74	49	44	47	81	10	16	30	61	78	76	72	53	57	29	14	40	61	38	16	83	35	100	75	75	100	87	77	12	25	65	28	95	38	53	59	30	85	39	20	27	94	70	79	78	16	43	33	88	48	82	28	28	68	20	13	74	36	80	39	11	65	90	53	55	76	48	58	36	47	99	33	75	50	61	48	16	22	25	66	68	28	32	86	20
69	85	70	86	46	92	48	95	86	94	32	58	22	21	41	50	14	92	41	71	83	39	97	29	90	21	91	80	22	51	37	55	86	41	22	44	61	72	91	64	80	26	57	59	45	22	17	48	11	24	83	85	25	57	13	52	39	23	17	11	85	63	58	84	23	64	33	77	91	12	58	60	31	90	89	44	39	72	42	83	38	28	46	26	40	59	71	82	66	69	71	30	68	21	72	67	61	36	73	21	44	55	72	17	22	99	27	69	88	58	83	26	79	47	97	79	21	64	96	82	57	56	35	23	61	17	36	43	46	47	80	61	34	96	21	22	43	79	28	24	55	87	78	10	21	24	41	94	99	80	90	44	48	46	85	62	53	77	52	75	97	64	25	95	10	21	53	89	66	91	53	44	13	91	93	20	85	67	96	96	59	13	57	15	24	68	54	47	53	22	10	12	79	42	45	54	46	31	72	44	81	52	83	60	100	43	23	66	100	18	35	32	37	42	55	94	16	30	76	18	92	52	85	71	48	36	25	36	13	94	84	86	57	22	49	98	68	45	37	65	63	24	68	70	80	93	26	66	60	49	40	70	28	58	97	85	97	93	19	87	14	31	99	60	60	47	69	82	25	85	98	98	13	39	48	15	12	31	70	60	77	55	50	14	64	69	49	30	92	48	49	43	25	23	77	58	84	57	26	69	78	51	64	25	67	48	79	95	34	47	32	69	27	58	72	62	29	31	56	58	83	43	64	99	80	89	84	14	12	100	92	73	26	59	95	25	26	72	64	82	21	31	27	51	98	59	17	57	46	42	98	85	62	11	29	93	85	64	14	25	97	87	49	99	79	44	90	56	41	91	24	98	93	81	38	98	92	52	77	85	77	81	36	63	26	24	51	49	42	25	74	38	11	50	22	57	78	21	69	20	42	44	45	69	43	45	29	86	40	98	68	18	16	38	76	49	51	83	76	15	59	61	17	17	65	73	73	55	61	78	95	79	50	55	59	46	72	46	13	40	56	91	75	60	17	46	96	87	52	13	43	84	52	53	14	93	68	47	55	51	58	33	54	68	40	46	71	88	44	60	94	39	43	38	63	41	21	18	33	99	34	97	42	64	91	68	34	87	31	36	74	99	66	85	36	64	20	74	38	47
90	39	35	59	96	86	29	29	10	51	66	40	42	97	18	63	72	73	35	12	67	91	16	18	70	62	17	84	100	39	37	51	27	34	54	65	43	13	84	11	23	46	99	51	87	68	57	32	78	96	58	91	46	12	79	13	85	84	66	57	32	25	29	91	33	25	78	25	96	62	14	36	50	89	26	96	28	66	79	55	91	82	75	90	29	91	55	55	21	19	99	67	80	61	40	91	38	71	67	28	48	52	41	70	44	76	45	16	91	84	53	32	88	83	16	99	99	22	86	10	96	51	77	82	47	91	59	19	44	37	30	27	87	83	16	81	12	34	76	28	47	79	66	57	56	55	50	44	33	37	99	72	99	73	31	31	78	12	32	94	93	35	47	53	86	45	34	77	94	32	80	98	33	79	15	95	60	90	64	16	72	40	32	17	97	14	46	99	64	84	89	65	72	48	66	25	80	48	89	86	49	20	49	47	31	90	47	35	40	61	22	88	62	97	32	92	32	97	21	15	81	96	64	18	16	63	95	51	57	63	92	18	52	29	21	90	33	30	92	11	72	15	14	25	48	89	12	40	64	15	88	48	19	60	49	34	54	15	55	52	21	55	77	56	85	66	15	54	14	90	92	58	27	82	67	16	60	68	96	28	95	71	49	37	72	14	83	42	55	98	58	77	11	29	27	83	51	83	16	71	56	36	40	99	15	30	86	15	74	67	40	24	78	77	24	24	47	92	81	45	73	97	15	85	53	42	50	22	54	87	50	82	83	62	90	74	47	80	72	74	58	19	99	51	54	15	85	26	43	23	20	35	41	93	28	38	89	86	18	97	70	30	88	89	62	94	100	39	84	79	69	55	52	57	64	64	96	20	66	80	96	89	80	73	66	12	36	41	75	13	20	34	85	18	84	91	55	83	92	78	66	33	90	37	27	74	96	48	49	88	32	20	80	22	92	49	83	45	47	57	48	98	16	13	92	71	99	13	23	40	70	37	70	18	17	77	27	61	82	85	79	23	31	55	30	92	70	72	26	38	72	94	45	21	95	98	50	70	99	93	30	66	93	66	94	17	54	52	52	21	12	14	86	76	13	48	67	49	96	75	28	48	32	52	19	19	61	31	89	93	16	30	55	91	43	98	90	44	42	98
78	64	78	68	25	25	76	21	74	19	61	32	76	22	49	12	13	89	65	78	62	98	73	17	63	14	100	60	79	61	32	62	63	97	27	40	56	25	98	42	20	98	63	45	78	15	66	23	19	74	19	71	21	26	46	54	34	16	71	27	33	95	98	42	39	14	96	59	34	31	75	97	13	46	82	36	28	52	22	45	13	61	91	19	46	91	72	45	76	60	37	69	31	97	26	26	74	47	39	66	93	27	79	56	37	51	85	67	58	35	29	37	76	76	78	54	49	91	23	76	58	16	22	37	96	74	71	23	31	43	56	29	52	33	47	91	16	20	99	86	91	85	58	85	40	80	58	16	61	39	88	11	79	39	87	91	60	91	26	90	82	30	94	47	36	38	81	21	65	88	73	42	32	95	23	26	54	99	13	82	82	53	94	98	81	60	66	52	32	42	53	60	82	42	94	10	95	97	64	86	82	75	10	36	66	34	36	77	24	18	56	25	44	17	36	66	24	48	26	96	30	45	98	52	26	73	45	47	61	52	81	31	33	85	15	13	17	32	88	68	38	30	64	73	34	41	50	62	16	33	83	95	31	35	77	66	27	16	17	100	40	71	67	34	73	97	50	56	69	67	48	89	51	34	43	66	94	80	20	75	94	76	81	81	86	19	49	91	53	32	58	54	92	24	88	95	16	45	61	69	100	35	87	81	65	73	87	18	60	71	45	61	90	81	85	75	27	55	69	76	100	82	54	15	85	18	71	92	66	50	85	96	78	86	15	13	29	21	75	97	20	40	95	98	87	61	38	56	19	19	57	62	98	86	49	48	31	33	55	92	46	90	26	22	56	86	83	13	79	37	80	65	55	79	50	19	59	17	36	58	79	23	77	30	22	24	63	12	27	28	16	32	23	62	29	86	28	81	10	34	37	65	78	39	45	36	58	55	72	35	89	32	68	84	62	74	21	93	82	29	99	48	86	52	51	84	77	86	40	56	84	54	55	68	17	35	99	36	31	61	66	64	63	67	40	12	70	57	50	54	41	24	71	68	82	43	32	22	79	83	91	32	40	20	47	33	53	87	90	21	27	44	40	92	41	34	55	61	76	16	36	93	24	81	81	34	21	64	94	51	22	35	71	86	67	15	40	38	14	48
85	16	44	94	11	77	77	83	59	51	87	74	17	61	57	98	49	30	39	29	34	91	47	42	69	32	47	80	11	62	80	24	96	56	86	14	67	49	57	34	56	34	38	11	25	91	34	42	45	73	53	68	18	25	42	55	51	50	42	86	80	87	57	98	100	69	22	46	53	13	23	50	24	81	86	27	89	57	93	10	76	33	95	34	52	81	76	75	20	52	14	78	70	11	29	83	69	100	30	40	42	81	40	64	25	13	68	44	85	93	25	59	48	99	15	38	78	70	68	80	11	14	79	24	59	19	70	88	63	72	11	46	75	90	29	94	83	47	58	73	94	80	96	88	47	28	48	78	66	95	92	78	47	17	50	51	29	33	65	13	82	89	95	26	83	24	96	65	61	92	18	69	44	95	77	73	51	16	62	90	84	49	41	67	15	35	53	88	99	39	50	56	64	18	63	42	78	64	77	28	35	44	10	29	64	32	12	97	65	82	59	96	34	79	73	80	95	62	24	86	67	54	75	13	40	28	84	26	22	35	17	23	22	57	49	33	58	17	81	40	28	26	34	72	17	79	25	87	19	54	92	80	56	25	18	74	83	20	65	98	69	12	96	43	89	47	89	60	40	55	99	18	23	12	53	88	11	50	26	47	22	19	46	63	84	96	37	68	48	24	72	87	60	77	70	98	80	62	69	30	46	29	41	19	98	35	95	55	82	73	66	67	13	59	20	36	68	80	89	82	64	79	19	59	37	47	50	97	18	77	97	72	99	35	88	63	81	41	89	57	43	58	99	15	71	46	19	77	38	98	96	68	82	100	27	93	67	33	77	56	46	69	68	71	13	53	66	37	21	45	91	83	97	95	20	26	54	82	81	40	31	88	76	85	35	96	98	53	73	48	74	35	11	85	43	67	44	67	34	39	70	83	73	60	48	84	69	62	97	17	24	18	45	71	58	62	43	43	77	97	96	11	39	42	59	100	53	87	99	11	48	26	77	92	51	24	88	13	92	65	60	49	60	91	79	59	31	84	50	31	70	93	55	57	42	54	63	93	35	85	19	79	48	67	88	41	77	78	19	55	36	22	43	31	87	96	38	62	23	80	51	32	57	43	20	60	85	52	80	87	95	54	32	15	40	75	22	75	88	55
41	89	62	45	66	35	33	26	84	20	28	42	61	46	40	42	76	42	14	36	62	23	41	37	52	64	30	63	16	83	77	31	61	57	64	86	46	90	32	48	62	94	58	40	39	61	45	31	17	74	89	30	75	69	16	81	89	96	21	38	61	11	78	95	58	15	37	46	47	37	70	64	56	54	14	55	47	22	21	32	87	58	78	64	92	91	31	64	48	62	16	21	40	23	66	53	87	34	70	64	79	19	88	93	84	43	13	70	39	56	54	38	57	21	70	19	32	31	17	98	71	67	47	54	56	66	88	98	11	43	63	36	96	44	96	97	23	58	62	23	62	60	99	61	41	86	94	49	47	86	57	92	10	39	54	50	77	98	77	68	80	60	23	59	76	65	21	35	83	88	13	63	85	84	40	97	78	92	24	21	55	78	47	87	13	48	95	81	30	21	20	56	93	30	20	15	50	95	77	63	36	59	24	59	73	15	21	50	39	92	96	96	71	43	25	45	95	39	51	82	63	47	99	16	14	48	18	77	60	44	21	35	39	70	77	25	19	95	76	28	24	36	89	11	65	86	58	53	30	15	17	59	19	81	96	26	18	50	21	56	49	54	75	92	24	23	62	23	41	25	64	43	67	91	96	91	97	43	20	86	82	30	96	32	53	16	97	75	11	32	93	13	30	36	72	85	19	53	34	60	60	42	48	57	33	36	89	83	30	74	97	55	19	98	18	27	60	91	55	87	63	35	13	36	55	88	72	29	32	54	31	44	41	41	95	100	67	71	24	65	29	66	93	37	48	81	55	73	13	42	19	31	78	34	17	68	78	72	38	44	31	15	97	41	83	91	71	13	69	15	50	71	65	34	73	24	90	47	72	31	27	99	49	19	29	53	25	52	89	13	60	13	56	61	65	26	56	65	56	53	36	71	38	85	84	95	29	95	92	78	68	84	77	12	61	36	95	60	63	54	79	38	69	55	15	23	45	55	78	56	95	75	37	44	69	38	48	81	56	89	49	52	41	21	99	97	39	63	23	86	42	44	19	45	55	88	88	64	51	94	87	22	34	45	69	58	73	46	33	33	57	20	45	34	70	86	80	49	45	78	22	98	57	26	28	45	16	84	86	28	18	12	14	38	13	45	18	88	45	93
55	17	75	53	34	84	39	62	79	22	50	58	94	57	97	21	73	34	12	34	23	82	76	35	41	78	75	89	36	95	87	73	12	48	47	70	41	44	21	50	83	67	11	83	60	24	11	20	96	89	75	71	87	55	33	38	51	29	35	36	52	95	58	45	19	16	26	29	80	15	98	40	57	35	12	82	45	21	60	63	58	23	28	73	22	39	29	45	90	63	46	78	57	92	35	41	29	52	18	80	63	73	53	63	76	64	49	18	55	74	87	98	82	15	84	56	68	77	56	15	33	38	40	91	94	96	72	21	50	64	47	35	57	29	85	68	43	22	82	19	73	10	19	61	54	17	29	48	87	87	95	59	56	29	39	48	85	67	36	86	14	71	95	38	52	91	81	17	52	30	43	92	93	30	81	70	76	79	64	65	87	88	47	19	37	96	40	27	67	28	33	80	13	51	72	78	90	97	69	70	87	12	10	43	18	38	36	36	43	79	50	98	51	59	42	51	32	60	53	93	48	20	59	27	38	15	98	32	97	12	11	49	56	78	87	85	81	21	12	91	96	28	35	65	86	71	80	55	61	56	86	44	86	35	79	73	88	66	98	46	31	28	14	22	82	71	50	70	59	83	58	55	88	36	52	12	65	49	80	61	81	34	86	86	36	35	100	22	80	93	89	83	94	39	43	22	41	99	75	25	43	50	54	74	75	71	20	98	57	79	98	92	71	21	35	57	96	92	76	56	64	64	98	53	25	45	34	33	48	21	64	55	59	84	46	37	17	43	61	20	95	79	41	34	55	67	14	84	36	93	17	77	16	65	96	24	50	91	35	71	87	45	54	27	69	43	40	84	70	61	66	81	78	98	23	98	67	43	89	69	11	31	48	24	69	15	63	54	94	74	12	25	60	97	79	23	96	58	46	11	17	52	30	54	55	40	39	68	61	13	19	25	56	26	56	12	55	78	29	11	81	87	34	66	51	19	99	27	78	12	44	64	42	38	86	76	50	41	57	80	44	47	90	60	40	76	76	52	66	20	40	31	30	36	94	71	17	63	56	36	28	87	91	83	64	89	22	35	25	41	91	53	93	92	98	89	53	27	27	16	98	53	65	17	60	35	28	46	86	74	49	98	94	45	15	63	77	60	62	66"""
task_data = [(j,i,int(v)) for i, line in enumerate(p_ij.split('\n')) for j, v in enumerate(line.split('\t')) ]

w_j = \
"""78	19	68	12	77	21	7	16	99	15	61	54	48	72	73	61	80	79	100	79	80	93	83	8	70	66	99	51	87	30	6	21	56	52	28	49	26	7	34	72	58	64	45	63	14	58	50	44	48	30	35	19	78	37	89	72	35	29	42	36	73	25	30	21	32	65	88	46	9	46	6	51	18	23	8	90	61	78	73	4	24	29	17	63	65	71	73	75	39	6	64	49	80	87	98	45	28	68	48	3	66	62	100	51	77	87	51	27	39	93	27	74	37	74	91	70	32	18	55	47	65	90	74	79	57	24	64	16	75	72	15	59	27	51	89	87	15	12	92	37	51	28	14	25	6	94	64	83	58	94	29	98	91	47	38	44	57	3	69	19	43	75	93	55	92	56	93	48	41	13	30	46	28	57	22	41	65	57	96	93	73	5	17	65	51	51	18	63	77	43	8	41	9	70	15	82	66	36	84	34	5	1	41	18	17	60	14	32	68	88	65	66	41	54	34	8	7	60	35	76	87	32	84	29	10	79	24	51	15	55	9	52	41	57	43	15	17	75	2	57	3	82	85	48	87	10	34	91	36	60	48	82	21	55	5	60	70	50	54	7	98	48	39	77	89	78	41	70	27	24	44	58	82	80	4	91	75	28	2	5	91	15	73	45	20	27	14	37	48	77	78	14	18	76	45	16	58	1	92	30	45	3	48	23	15	81	72	15	88	17	37	22	26	31	30	90	68	10	13	60	48	56	58	34	82	3	18	52	62	87	31	14	47	5	80	88	5	12	15	48	41	77	55	56	97	8	56	26	39	66	47	60	18	4	66	44	86	15	10	85	67	20	64	23	33	43	85	47	20	5	79	41	90	33	67	22	46	64	40	31	41	55	3	45	14	59	80	1	87	4	36	50	27	27	39	38	20	83	90	30	3	29	64	64	95	93	47	50	25	29	3	79	52	48	84	92	36	35	79	32	34	53	55	43	13	28	16	39	52	58	16	94	74	96	81	20	13	31	55	45	59	95	53	10	80	29	81	75	27	99	45	32	96	84	8	30	44	57	77	68	82	78	98	93	86	38	62	50	23	53	93	79	85	30	90	24	99	73	23	3	54	46	16	31	17	71	50	89	45	46	75	96	3"""

In [3]:
# Data.
task_data_2 = [  # (task_id, machine_id, processing_time)
    (0, 0, 3), (0, 1, 2), (0, 2, 2),  # Task0
    (1, 0, 2), (1, 1, 1), (1, 2, 2),  # Task1
    (2, 0, 6), (2, 1, 1),  # Task2
    (3, 0, 1) # Task3
]

In [4]:
print(task_data)

[(0, 0, 97), (1, 0, 96), (2, 0, 27), (3, 0, 39), (4, 0, 52), (5, 0, 67), (6, 0, 80), (7, 0, 82), (8, 0, 97), (9, 0, 71), (10, 0, 52), (11, 0, 92), (12, 0, 44), (13, 0, 43), (14, 0, 43), (15, 0, 46), (16, 0, 20), (17, 0, 82), (18, 0, 62), (19, 0, 92), (20, 0, 60), (21, 0, 72), (22, 0, 25), (23, 0, 65), (24, 0, 20), (25, 0, 41), (26, 0, 14), (27, 0, 24), (28, 0, 94), (29, 0, 75), (30, 0, 63), (31, 0, 80), (32, 0, 60), (33, 0, 71), (34, 0, 51), (35, 0, 53), (36, 0, 71), (37, 0, 45), (38, 0, 49), (39, 0, 63), (40, 0, 79), (41, 0, 24), (42, 0, 20), (43, 0, 93), (44, 0, 42), (45, 0, 45), (46, 0, 71), (47, 0, 21), (48, 0, 18), (49, 0, 65), (50, 0, 40), (51, 0, 30), (52, 0, 10), (53, 0, 68), (54, 0, 67), (55, 0, 17), (56, 0, 73), (57, 0, 39), (58, 0, 78), (59, 0, 46), (60, 0, 42), (61, 0, 55), (62, 0, 66), (63, 0, 90), (64, 0, 63), (65, 0, 100), (66, 0, 43), (67, 0, 48), (68, 0, 53), (69, 0, 71), (70, 0, 24), (71, 0, 21), (72, 0, 28), (73, 0, 76), (74, 0, 87), (75, 0, 26), (76, 0, 43), (77, 0,

# Find gready solution for the bounds

In [5]:
max_task = max([task[0] for task in task_data])
max_resources = max([task[1] for task in task_data])

resource_times = {key: 0 for key in range(max_resources+1)}

for i in range(max_task+1):
    feasible_resources = list(filter(lambda t : t is not None, [task if task[0]==i else None for task in task_data]))
    min_resource_time, min_resource = math.inf, None
    for j, feasible_resource in enumerate(feasible_resources):
        rt = resource_times[feasible_resource[1]] + feasible_resource[2]
        if rt < min_resource_time:
            min_resource_time, min_resource = rt, feasible_resource[1]
    #print(i, min_resource, min_resource_time)
    resource_times[min_resource] = min_resource_time

print(resource_times)
greedy_resource_time = max(resource_times.values())
print(greedy_resource_time)

{0: 137, 1: 146, 2: 137, 3: 144, 4: 138, 5: 135, 6: 143, 7: 138, 8: 141, 9: 152, 10: 140, 11: 138, 12: 142, 13: 141, 14: 151, 15: 135, 16: 137, 17: 143, 18: 147, 19: 141, 20: 140, 21: 142, 22: 143, 23: 146, 24: 140, 25: 138, 26: 141, 27: 148, 28: 142, 29: 148, 30: 138, 31: 143, 32: 144, 33: 146, 34: 131, 35: 146, 36: 146, 37: 142, 38: 130, 39: 135, 40: 140, 41: 141, 42: 139, 43: 133, 44: 146, 45: 140, 46: 135, 47: 141, 48: 151, 49: 134}
152


# Create CP model

In [6]:
# Create the model.

class UnrelatedMachinesScheduling:
    def __init__(self, task_data, max_value=None):
        self.task_data = task_data
        self.__define_model(max_value)
        self.__define_constraints()
        self.__define_objective()

    def __define_model(self, max_value):
        self.model = cp_model.CpModel()
        # Named tuple to store information about created variables.
        self.task_type = collections.namedtuple("task_type", "start end interval assigned")
        # Creates job intervals and add to the corresponding machine lists.
        self.all_tasks = {}
        self.task_assigned_machines = collections.defaultdict(list)
        self.machines = collections.defaultdict(list)
        self.intervals = collections.defaultdict(list)
        self.goal_variables = collections.defaultdict(list)
        if max_value:
            self.horizon = self.__get_horizon()[0], max_value
        else:
            self.horizon = self.__get_horizon()
        print(self.horizon)

    def __get_horizon(self):
        max_task = max([task[0] for task in self.task_data])
        max_resource = max([task[1] for task in self.task_data])
        max_resource_duration = dict()
        min_resource_duration = dict()
        min_task_duration, max_task_duration = dict(), dict()
        for t in range(max_task+1):
            #min_resource_duration[r] = min([task[2] if task[1]==r else math.inf for task in self.task_data])
            #max_resource_duration[r] = max([task[2] if task[1]==r else 0 for task in self.task_data])
            min_task_duration[t] = min([task[2] if task[0]==t else math.inf for task in self.task_data])
            max_task_duration[t] = max([task[2] if task[0]==t else 0 for task in self.task_data])
        #horizon = max(min_task_duration.values()), sum(min_task_duration.values())+1#, sum(max_task_duration.values())
        horizon = int(sum(min_task_duration.values())/(max_resource+1)+1), sum(min_task_duration.values())+1
        return horizon
    
    
    def __define_constraints(self):
        #Define decision variable
        for task, machine, processing_time in self.task_data:
            suffix = f"_{task}_{machine}"
            assigned_var = self.model.NewBoolVar('assigned' + suffix)
            self.task_assigned_machines[task].append(assigned_var)
            start_var = self.model.NewIntVar(0, self.horizon[1], 'start' + suffix)
            end_var = self.model.NewIntVar(0, self.horizon[1], 'end' + suffix)
            interval_var = self.model.NewOptionalIntervalVar(
                start    = start_var,
                end      = end_var,
                size = processing_time,
                is_present = assigned_var,
                name = 'interval'+suffix
            )
            goal_variable = self.model.NewIntVar(0, self.horizon[1], 'goal' + suffix)
            #self.model.Add(goal_variable == end_var).OnlyEnforceIf(assigned_var)
            #self.model.Add(goal_variable == 0).OnlyEnforceIf(assigned_var.Not())
            self.model.AddMultiplicationEquality(goal_variable, assigned_var, end_var)
            self.goal_variables[task, machine] = goal_variable
            t = self.task_type(
                start = start_var,
                end = end_var,
                assigned = assigned_var,
                interval = interval_var
            )
            self.intervals[task, machine] = t 
            self.machines[machine].append(t)

        #Constraint 1: No machine may work on more than one task simultaneously
        machines_count = 1 + max(task[1] for task in self.task_data)
        all_machines = range(machines_count)
        for machine in all_machines:
            #no overlap between intervals of one machine
            self.model.AddNoOverlap([t.interval for t in self.machines[machine]])

        #Constraint 2: All tasks must be assigned to exactly one machine
        tasks_count = 1+ max(task[0] for task in self.task_data)
        all_tasks = range(tasks_count)
        for task in all_tasks:
            #sum of assignment variable of task must be one
            self.model.Add(cp_model.LinearExpr.Sum(self.task_assigned_machines[task]) == 1)

        #Optimization Constraint 3: No gaps between tasks on one machine
        """
        for i, machine_tasks in enumerate(self.machines.values()):
            print(i)
            for t1 in machine_tasks:
                iv = self.model.NewBoolVar(str(t1)+"0")
                self.model.Add(t1.start == 0).OnlyEnforceIf(iv)
                literals = [iv]
                for t2 in machine_tasks:
                    if t1 != t2:
                        iv = self.model.NewBoolVar(str(t1)+str(t2))
                        self.model.Add(t1.start == t2.end).OnlyEnforceIf(iv)
                        literals.append(iv)
                self.model.AddBoolXOr(literals)
        """
    
    def __define_objective(self):
        # Makespan objective.
        obj_var = self.model.NewIntVar(self.horizon[0], self.horizon[1], "makespan")
        self.model.AddMaxEquality(
            obj_var,
            [self.goal_variables[task, machine] for task, machine, _ in self.task_data],
        )
        self.model.Minimize(obj_var)

    def solve(self, solver=cp_model.CpSolver()):
        status = solver.Solve(self.model)
        return (solver, status)


model = UnrelatedMachinesScheduling(task_data, max_value=greedy_resource_time)

(118, 152)


In [ ]:
# Creates the solver and solve.
start_time = time.time()
logging.basicConfig(level=logging.INFO, filename="log.txt", filemode="w")
solver = cp_model.CpSolver()
solver.parameters.log_search_progress = True
solver.log_callback = logging.info
status = solver.Solve(model.model)
end_time = time.time()


Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true
Setting number of workers to 32

Initial optimization model '': (model_fingerprint: 0x24b1cb956637f916)
#Variables: 100'001 (#ints: 1 in objective)
  - 25'000 Booleans in [0,1]
  - 75'000 in [0,152]
  - 1 in [118,152]
#kIntProd: 25'000
#kInterval: 25'000 (#enforced: 25'000)
#kLinMax: 1 (#expressions: 25'000)
#kLinear2: 25'000 (#enforced: 25'000)
#kLinearN: 500 (#terms: 25'000)
#kNoOverlap: 50 (#intervals: 25000, #optional: 25000)

Starting presolve at 0.02s
  1.71e-02s  0.00e+00d  [DetectDominanceRelations] 
  1.44e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  5.21e-04s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=500 
  9.10e+00s  1.00e+00d *[Probe] #probed=87'378 #new_bounds=25'000 #new_binary_clauses=897'940 
  1.63e-03s  0.00e+00d  [MaxClique] 
  1.79e-02s  0.00e+00d  [DetectDominanceRelations] 
  1.67e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual

In [ ]:
print('feasible:', status == cp_model.FEASIBLE, ', optimal:', status == cp_model.OPTIMAL)
print(end_time - start_time)
print(solver.ObjectiveValue())
for iid, interval in sorted(model.intervals.items(), key=lambda s: s[0][::-1]):
    if solver.Value(interval.assigned):
        print(iid, f"{solver.Value(interval.start)}-{solver.Value(interval.end)}")